Housekeeping

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.optimize import fsolve
import itertools
from astropy.table import Table

from f_detrend import detrend_fuction
pd.set_option('display.max_rows', None)

sn.set_style('whitegrid')
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Auxiliary functions

In [5]:
def read_table(filename,keyword='ciclo',rescale=1,ncols=5,start=2,end_row='Total',
               endword=None,omit_list=['t20_21'],index_to_int=True,debug=False):
    t0 = start
    t1 = ncols+start
    results_follow = 0
    row_names = []
    interaction = 0
    f = open(filename,'r')
    for line in f:
        words = line.split()
        if debug:
            print(words)
        if len(words)>1 and words[0] == keyword:
            results_follow=1
            if endword==None:
                colnames = words[t0:t1]
            else:
                buff_v = []
                for w in words[t0:]:
                    if w==endword:
                        break
                    else:
                        buff_v.append(w)
                colnames = buff_v[:-1]
                t1 = len(buff_v)+t0-1
            table_raw = np.empty(t1-t0)
        elif len(words)>1 and words[0] == end_row:
            break
        elif len(words) == 1:
            interaction = 0
            i_count = 0
        elif len(words)>1 and results_follow==1: # and words[0] not in omit_list
            if (len(words)==2 and words[1]=="|") or (len(words)==3 and words[2]=="|"):
                var_name = words[0]
                interaction = 1
                i_count = 0
            else:
                if interaction==0:
                    row_names.append(words[0])
                else:
                    end_idx = words.index("|")
                    new_name = var_name+"_{}"*end_idx
                    new_name = new_name.format(*words[:end_idx])
                    row_names.append(new_name)                 
                if words[2]=='|':
                    t00= t0+1
                    t11 = t1+1
                elif words[3]=='|':
                    t00= t0+2
                    t11 = t1+2
                else:
                    t00 = t0
                    t11 = t1

                buff_v = []
                for w in words[t00:t11]: 
                    w = w.replace(",","")
                    buff_v.append(w)
                table_raw = np.vstack((table_raw,np.array(buff_v,dtype=float)/rescale))
        
    table_raw = table_raw[1:,:]
    if debug:
        print(table_raw.shape)
        print(colnames)
    if index_to_int:
        return pd.DataFrame(table_raw, columns=colnames,index=np.array(row_names,dtype=int))
    else:
        return pd.DataFrame(table_raw, columns=colnames,index=np.array(row_names))
    
    
def read_descriptive_stats(filename,keyword_data,keyword_col,ncols,rownames,start=2,debug=False):
    t0 = start
    t1 = ncols+start
    table_raw = np.empty(t1-t0)
    f = open(filename,'r')
    for line in f:
        words = line.split()
        if debug:
            print(words)
        if len(words)>1 and words[0] == keyword_data:
            buff_v = []
            for w in words[t0:t1]: 
                w = w.replace(",","")
                buff_v.append(w)
            table_raw = np.vstack((table_raw,np.array(buff_v,dtype=float)))
        elif len(words)>1 and words[0] == keyword_col:
            colnames = words[t0:t1]

    return pd.DataFrame(table_raw[1:,:], columns=colnames, index=rownames)
 
    
def file_split(filename,path,startwords=['Summary',],endword="Total",header=None,Noobs=False):
    f = open(path+filename,'r')
    parent_file = filename[:filename.find('.')]
    di = 1
    R2s = []
    writing = 0
    for line in f:
        words = line.split()
        if len(words)>0 and 'Number' in words:
            noobs = words[-1]
        if len(words)>0 and 'R2' in words:
            R2s.append(words[-1])
        if len(words)>0 and words[0] in startwords:
            sf = open(path+parent_file+'_{}.log'.format(di), "w")
            writing = 1
            if header!=None:
                sf.write(header)
        elif len(words)>0 and words[0]==endword and writing:
            sf.write(line)
            writing = 0
            sf.close()
            di+=1
        if writing:
            sf.write(line)
    if Noobs==True:
        return (noobs,R2s)
            
def normalise_table(table):
    cols = table.columns
    table['Total'] = table.sum(axis=1).copy()
    for col in cols:
        table[col] = table[col]/table['Total']
    return table   

In [3]:
dates = []
for year in range(1987,2022):
    for quarter in range(1,5):
        dates.append(str(year)+"Q"+str(quarter))
dates = dates[1:]

t0 = 59
t05 = 71
T = len(dates)
date_dict = dict(zip(range(t0,T+t0),dates))
date_dict_reverse = dict(zip(dates,range(0,T)))
date_dict_reverse_ciclo = dict(zip(dates,range(t0,T+t0)))

time_periods_thin = []
for y in range(2006,2022):
    time_periods_thin.append(str(y))
    
time_periods = ["t08_11", "t11_14", "t14_17", "t17_20", "t20","t21"]
period_labels = ['2008-2011','2011-2014','2014-2017','2017-2020','2020','2021']
    
def rename_interaction(tab,int_subs):
    tab_chunk_1 = tab.iloc[0:7].copy()
    tab_chunk_2 = tab.iloc[7:].copy()
    tab_chunk_1.rename(index=int_subs,inplace=True)
    tab = pd.concat((tab_chunk_1,tab_chunk_2))
    return tab

In [7]:
path = './tables/'
omits = ['sexo1#c.ttrend','sexo1#c.ttrend2',"covid#sexo1","sexo1#occgroup","covid#sexo1#occgroup","sexo1#c.urate","occgroup","covid#occgroup"]

name = "sqtreg_table_advanced_simple_age3040__10_agefix"
N, r2s = file_split('{}.log'.format(name),path,startwords=['q25','q50','q75'],endword="_cons",
           header="ten_y | Coefficient std t p_stat lower_95 upper_95",Noobs=True)

tab_moms_25 = read_table(path+'{}_1.log'.format(name),
                         keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)
tab_moms_50 = read_table(path+'{}_2.log'.format(name),
                         keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)
tab_moms_75 = read_table(path+'{}_3.log'.format(name),
                         keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)


In [27]:
new_names_var = ['$t$', '$t$ x female', 'unemployment rate x male',
       'unemployment rate x female', 'female', 'age 35-40', 'age 35-40 x female',
       'occ group 2','occ group 3','occ group 4','occ group 5',
       '$\delta_{2020}$ x occ group 1', '$\delta_{2020}$ x occ group 2',
       '$\delta_{2020}$ x occ group 3', '$\delta_{2020}$ x occ group 4',
       '$\delta_{2020}$ x occ group 5', '$\delta_{2021}$ x occ group 1',
       '$\delta_{2021}$ x occ group 2', '$\delta_{2021}$ x occ group 3',
       '$\delta_{2021}$ x occ group 4', '$\delta_{2021}$ x occ group 5',
       '$\delta_{2022}$ x occ group 1', '$\delta_{2022}$ x occ group 2',
       '$\delta_{2022}$ x occ group 3', '$\delta_{2022}$ x occ group 4',
       '$\delta_{2022}$ x occ group 5',
        'occ group 2 x female','occ group 3 x female','occ group 4 x female','occ group 5 x female',     
       '$\delta_{2020}$ x occ group 1 x female',
       '$\delta_{2020}$ x occ group 2 x female',
       '$\delta_{2020}$ x occ group 3 x female',
       '$\delta_{2020}$ x occ group 4 x female',
       '$\delta_{2020}$ x occ group 5 x female',
       '$\delta_{2021}$ x occ group 1 x female',
       '$\delta_{2021}$ x occ group 2 x female',
       '$\delta_{2021}$ x occ group 3 x female',
       '$\delta_{2021}$ x occ group 4 x female',
       '$\delta_{2021}$ x occ group 5 x female',
       '$\delta_{2022}$ x occ group 1 x female',
       '$\delta_{2022}$ x occ group 2 x female',
       '$\delta_{2022}$ x occ group 3 x female',
       '$\delta_{2022}$ x occ group 4 x female',
       '$\delta_{2022}$ x occ group 5 x female', '$\\beta_0$']

# reorder_cols = [r'$\beta_0$','female',r'partner tenure (years)',r'$t$',r'$t^{2}$',r'$t$ x female',r'$t^2$ x female',
#                      r'$\delta_{2020}$','$\delta_{2021}$',r'$\delta_{2020}$ x female',
#                      r'$\delta_{2021}$ x female']

def rename_results(df,old_names,new_names,order):
    
    df.rename(columns=dict(zip(old_names,new_names)),inplace='True')
    df = df.reindex(columns=order)
    return df

def pass_regression_to_latex_column(df):

    coefs = df["Coefficient"].copy()
    names = coefs.index.copy()
    N = names.size

    coefs = coefs.apply(lambda x: "{:,.3f}".format(float(x)))
    coefs = coefs.rename(index=(dict(zip(names,np.arange(0,N*2,2)))))

    stds = df['std'].copy()
    stds = stds.apply(lambda x: "({:,.4f})".format(float(x)))
    stds = stds.rename(index=(dict(zip(names,np.arange(1,N*2,2)))))

    final_table = pd.concat([coefs, stds]).sort_index(kind='merge')
    final_table = final_table.rename(index=dict(zip(np.arange(0,N*2,2),names)))
    final_table = final_table.rename(index=dict(zip(np.arange(1,N*2,2),[""]*N)))
    
    return final_table

In [28]:
res_names = ['25th pctl','50th pctl','75th pctl']

all_results = [tab_moms_25,tab_moms_50,tab_moms_75]
reorder_cols = new_names_var


for i in range(len(all_results)):
    all_results[i] = rename_results(all_results[i].T,all_results[i].index,new_names_var,new_names_var)
table = pd.DataFrame(pass_regression_to_latex_column(all_results[0].T),columns=[res_names[0],])
for ir,res in enumerate(all_results[1:]):
    col = pass_regression_to_latex_column(res.T)
    table[res_names[ir+1]] = col.values
    
    
for i,reg in enumerate(res_names):
    for var in reorder_cols:
        if all_results[i].loc['p_stat'][var] < 0.1 and all_results[i].loc['p_stat'][var] > 0.05:
            table[reg].loc[var] = r"${}^*$".format(table[reg].loc[var])
        elif all_results[i].loc['p_stat'][var] < 0.05 and all_results[i].loc['p_stat'][var] > 0.01:
            table[reg].loc[var] = r"${}^{{**}}$".format(table[reg].loc[var])
        elif all_results[i].loc['p_stat'][var] < 0.01:
            table[reg].loc[var] = r"${}^{{***}}$".format(table[reg].loc[var])
        else:
            table[reg].loc[var] = r"${}$".format(table[reg].loc[var])
            
N_row = pd.DataFrame([N]*3, index=res_names, columns=['N',]).T
R2_row = pd.DataFrame(r2s, index=res_names, columns=[r'Pseudo $R^2$',]).T
table = pd.concat([table,N_row,R2_row])

# print(table.loc[[r'$\delta_{2020}$','$\delta_{2021}$',r'$\delta_{2020}$ x female',
#                      r'$\delta_{2021}$ x female']].to_latex(escape=False))
print(table)
table.to_latex('./tables/qtregs_occ_3040_full.tex',escape=False)

                                             25th pctl       50th pctl  \
$t$                                     $-0.011^{***}$  $-0.007^{***}$   
                                              (0.0007)        (0.0014)   
$t$ x female                             $0.012^{***}$   $0.038^{***}$   
                                              (0.0009)        (0.0019)   
unemployment rate x male                 $0.034^{***}$   $0.046^{***}$   
                                              (0.0029)        (0.0040)   
unemployment rate x female               $0.055^{***}$   $0.019^{***}$   
                                              (0.0020)        (0.0034)   
female                                        $-0.063$    $0.191^{**}$   
                                              (0.0884)        (0.0829)   
age 35-40                                $0.781^{***}$   $2.353^{***}$   
                                              (0.0306)        (0.0405)   
age 35-40 x female                    

<ipython-input-28-d1098afc94af>:33: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  table.to_latex('./tables/qtregs_occ_3040_full.tex',escape=False)


In [30]:
IoI_20,IoI_21,IoI_22 = [], [], []
std_idx_20, std_idx_21, std_idx_22 = [], [], []
for i,idx in enumerate(table.index):
    if str(idx)[:13]=="$\delta_{2020":
        if idx.split(' x ')[-1]=='female':
            IoI_20.append(idx)
            std_idx_20.append(i+1)
    elif str(idx)[:13]=="$\delta_{2021":
        if idx.split(' x ')[-1]=='female':
            IoI_21.append(idx)
            std_idx_21.append(i+1)
    elif str(idx)[:13]=="$\delta_{2022":
        if idx.split(' x ')[-1]=='female':
            IoI_22.append(idx)
            std_idx_22.append(i+1)

table_dummies_std_f_20,table_dummies_std_f_21, table_dummies_std_f_22 = table.iloc[std_idx_20,:].copy(),table.iloc[std_idx_21,:].copy(),table.iloc[std_idx_22,:].copy()
table_dummies_f_20,table_dummies_f_21, table_dummies_f_22 = table.loc[IoI_20,:].copy(),table.loc[IoI_21,:].copy(),table.loc[IoI_22,:].copy()

# IoI_20[1] = IoI_20[1].replace('Manifactures','Manufactures')
# IoI_20[3] = IoI_20[3].replace('manifuctures','Manufactures')

table_dummies_std_f_20['coefs'] = IoI_20
table_dummies_std_f_21['coefs'] = IoI_21
table_dummies_std_f_22['coefs'] = IoI_22
table_dummies_std_f_20.set_index('coefs',inplace=True)
table_dummies_std_f_21.set_index('coefs',inplace=True)
table_dummies_std_f_22.set_index('coefs',inplace=True)

all_data = [table_dummies_f_20.T,table_dummies_f_21.T,table_dummies_f_22.T]
all_stds = [table_dummies_std_f_20.T,table_dummies_std_f_21.T,table_dummies_std_f_22.T]

One_table = {}
for i in range(3):
    coefs, stds = all_data[i], all_stds[i]
    cols= coefs.keys()
    New_table = coefs[cols[0]]
    New_table = pd.concat((New_table.rename(cols[0][18:]),stds[cols[0]].rename(cols[0][18:]+'_std')),axis=1)
    for name in cols[1:]:
        New_table = pd.concat((New_table,coefs[name].rename(name[18:]),stds[name].rename(name[18:]+'_std')),axis=1)
    
    One_table[2020+i] = New_table.T.copy()
    
Final_table = pd.concat((One_table[2020]['25th pctl'].rename("2020"),
           One_table[2021]['25th pctl'].rename("2021"),
          One_table[2022]['25th pctl'].rename("2022")),axis=1)

for ind in Final_table.index[1::2]:
    Final_table.rename({ind:""}, inplace=True)
Final_table.replace({np.nan:'--'}, inplace=True)

Final_table2 = pd.concat((One_table[2020]['50th pctl'].rename("2020"),
           One_table[2021]['50th pctl'].rename("2021"),
          One_table[2022]['50th pctl'].rename("2022")),axis=1)

for ind in Final_table2.index[1::2]:
    Final_table2.rename({ind:""}, inplace=True)
Final_table2.replace({np.nan:'--'}, inplace=True)

Final_table3 = pd.concat((One_table[2020]['75th pctl'].rename("2020"),
           One_table[2021]['75th pctl'].rename("2021"),
          One_table[2022]['75th pctl'].rename("2022")),axis=1)

for ind in Final_table3.index[1::2]:
    Final_table3.rename({ind:""}, inplace=True)
Final_table3.replace({np.nan:'--'}, inplace=True)

Final_final_table = pd.concat((Final_table,Final_table2,Final_table3),axis=1,keys=['25th percentile','50th percentile','75th percentile'])
Final_final_table.loc["N",:] = table.loc['N'][0]
print(Final_final_table)
# Final_final_table.to_latex('./tables/qtregs_ind_3040_text.tex',escape=False)

                     25th percentile                                  \
                                2020            2021            2022   
occ group 1 x female        $-0.182$        $-0.260$  $-0.935^{***}$   
                            (0.2070)        (0.2759)        (0.1739)   
occ group 2 x female      $-0.418^*$  $-0.708^{***}$  $-1.096^{***}$   
                            (0.2360)        (0.2595)        (0.2716)   
occ group 3 x female  $-0.615^{***}$  $-0.783^{***}$         $0.314$   
                            (0.2279)        (0.2414)        (0.2723)   
occ group 4 x female        $-0.151$        $-0.298$  $-0.840^{***}$   
                            (0.2250)        (0.2295)        (0.2013)   
occ group 5 x female  $-0.466^{***}$  $-0.775^{***}$  $-0.820^{***}$   
                            (0.0811)        (0.1025)        (0.1952)   
N                            409,631         409,631         409,631   

                     50th percentile                           

In [31]:
IoI_20,IoI_21,IoI_22 = [], [], []
std_idx_20, std_idx_21, std_idx_22 = [], [], []
for i,idx in enumerate(table.index):
    if str(idx)[:13]=="$\delta_{2020":
        if idx.split(' x ')[-1]!='female':
            IoI_20.append(idx)
            std_idx_20.append(i+1)
    elif str(idx)[:13]=="$\delta_{2021":
        if idx.split(' x ')[-1]!='female':
            IoI_21.append(idx)
            std_idx_21.append(i+1)
    elif str(idx)[:13]=="$\delta_{2022":
        if idx.split(' x ')[-1]!='female':
            IoI_22.append(idx)
            std_idx_22.append(i+1)
table_dummies_std_f_20,table_dummies_std_f_21, table_dummies_std_f_22 = table.iloc[std_idx_20,:].copy(),table.iloc[std_idx_21,:].copy(),table.iloc[std_idx_22,:].copy()
table_dummies_f_20,table_dummies_f_21, table_dummies_f_22 = table.loc[IoI_20,:].copy(),table.loc[IoI_21,:].copy(),table.loc[IoI_22,:].copy()

# IoI_20[1] = IoI_20[1].replace('Manifactures','Manufactures')
# IoI_20[3] = IoI_20[3].replace('manifuctures','Manufactures')

table_dummies_std_f_20['coefs'] = IoI_20
table_dummies_std_f_21['coefs'] = IoI_21
table_dummies_std_f_22['coefs'] = IoI_22
table_dummies_std_f_20.set_index('coefs',inplace=True)
table_dummies_std_f_21.set_index('coefs',inplace=True)
table_dummies_std_f_22.set_index('coefs',inplace=True)

all_data = [table_dummies_f_20.T,table_dummies_f_21.T,table_dummies_f_22.T]
all_stds = [table_dummies_std_f_20.T,table_dummies_std_f_21.T,table_dummies_std_f_22.T]

One_table = {}
for i in range(3):
    coefs, stds = all_data[i], all_stds[i]
    cols= coefs.keys()
#     print(cols)
    New_table = coefs[cols[0]]
    New_table = pd.concat((New_table.rename(cols[0][18:]),stds[cols[0]].rename(cols[0][18:]+'_std')),axis=1)
    for name in cols[1:]:
        New_table = pd.concat((New_table,coefs[name].rename(name[18:]),stds[name].rename(name[18:]+'_std')),axis=1)
        
    One_table[2020+i] = New_table.T.copy()
    
Final_table = pd.concat((One_table[2020]['25th pctl'].rename("2020"),
           One_table[2021]['25th pctl'].rename("2021"),
          One_table[2022]['25th pctl'].rename("2022")),axis=1)

for ind in Final_table.index[1::2]:
    Final_table.rename({ind:""}, inplace=True)
Final_table.replace({np.nan:'--'}, inplace=True)

Final_table2 = pd.concat((One_table[2020]['50th pctl'].rename("2020"),
           One_table[2021]['50th pctl'].rename("2021"),
          One_table[2022]['50th pctl'].rename("2022")),axis=1)

for ind in Final_table2.index[1::2]:
    Final_table2.rename({ind:""}, inplace=True)
Final_table2.replace({np.nan:'--'}, inplace=True)

Final_table3 = pd.concat((One_table[2020]['75th pctl'].rename("2020"),
           One_table[2021]['75th pctl'].rename("2021"),
          One_table[2022]['75th pctl'].rename("2022")),axis=1)

for ind in Final_table3.index[1::2]:
    Final_table3.rename({ind:""}, inplace=True)
Final_table3.replace({np.nan:'--'}, inplace=True)

Final_final_table_b = pd.concat((Final_table,Final_table2,Final_table3),axis=1,keys=['25th percentile','50th percentile','75th percentile'])
# print(Final_final_table_b.to_latex(escape=False))

Final_final_final_table = pd.concat((Final_final_table_b,Final_final_table))
Final_final_final_table.to_latex('./tables/sqtreg_advanced_age3040_10_agefix_format.tex',escape=False,column_format='l|ccc|ccc|ccc|')
print(Final_final_final_table)

                     25th percentile                                  \
                                2020            2021            2022   
occ group 1           $-0.741^{***}$  $-0.809^{***}$        $-0.050$   
                            (0.1535)        (0.1630)        (0.2040)   
occ group 2                  $0.000$         $0.236$       $0.424^*$   
                            (0.1239)        (0.2035)        (0.2321)   
occ group 3            $0.825^{***}$   $0.807^{***}$        $-0.022$   
                            (0.2226)        (0.2196)        (0.2381)   
occ group 4            $0.436^{***}$   $0.663^{***}$   $0.603^{***}$   
                            (0.0932)        (0.0943)        (0.1329)   
occ group 5            $0.472^{***}$   $0.695^{***}$   $0.777^{***}$   
                            (0.0780)        (0.0829)        (0.1763)   
occ group 1 x female        $-0.182$        $-0.260$  $-0.935^{***}$   
                            (0.2070)        (0.2759)        (0.1

<ipython-input-31-50c9515a01ed>:72: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Final_final_final_table.to_latex('./tables/sqtreg_advanced_age3040_10_agefix_format.tex',escape=False,column_format='l|ccc|ccc|ccc|')


In [34]:
IoI_rest = []
std_idx_rest = []
for i,idx in enumerate(table.index):
    if str(idx)[:12]!="$\delta_{202" and str(idx)!='':
        
        IoI_rest.append(idx)
        if str(idx) in ['N',"Pseudo $R^2$"]:
            pass
        else:
            std_idx_rest.append(i+1)

table_rest_std_f_20 = table.iloc[std_idx_rest,:].copy()
table_rest_f_20 = table.loc[IoI_rest,:].copy()

table_rest_f_20 = table_rest_f_20.T

table_rest_std_f_20['coefs'] = IoI_rest[:-2]
table_rest_std_f_20.set_index('coefs',inplace=True)
table_rest_std_f_20 = table_rest_std_f_20.T

cols= table_rest_f_20.keys()[:-2]
# print(cols)
New_table = table_rest_f_20[cols[0]]
New_table = pd.concat((New_table.rename(cols[0]),table_rest_std_f_20[cols[0]].rename(cols[0]+'_std')),axis=1)
for name in cols[1:]:
    New_table = pd.concat((New_table,table_rest_f_20[name].rename(name),
                         table_rest_std_f_20[name].rename(name+'_std')),axis=1)


Final_table =  New_table.T.copy()

for ind in Final_table.index[1::2]:
    Final_table.rename({ind:""}, inplace=True)
Final_table.replace({np.nan:'--'}, inplace=True)
Final_table.loc['N',:] = table.loc['N',:]
Final_table.loc["Pseudo $R^2$",:] = table.loc["Pseudo $R^2$",:]
Final_table.to_latex('./tables/sqtreg_advanced_age3040_10_agefix_part1.tex',escape=False,column_format='l|ccc|')
print(Final_table)

                                 25th pctl       50th pctl       75th pctl
$t$                         $-0.011^{***}$  $-0.007^{***}$   $-0.004^{**}$
                                  (0.0007)        (0.0014)        (0.0019)
$t$ x female                 $0.012^{***}$   $0.038^{***}$   $0.026^{***}$
                                  (0.0009)        (0.0019)        (0.0023)
unemployment rate x male     $0.034^{***}$   $0.046^{***}$         $0.004$
                                  (0.0029)        (0.0040)        (0.0038)
unemployment rate x female   $0.055^{***}$   $0.019^{***}$  $-0.028^{***}$
                                  (0.0020)        (0.0034)        (0.0030)
female                            $-0.063$    $0.191^{**}$   $0.538^{***}$
                                  (0.0884)        (0.0829)        (0.0763)
age 35-40                    $0.781^{***}$   $2.353^{***}$   $3.481^{***}$
                                  (0.0306)        (0.0405)        (0.0526)
age 35-40 x female       

<ipython-input-34-78c905b3ad31>:37: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Final_table.to_latex('./tables/sqtreg_advanced_age3040_10_agefix_part1.tex',escape=False,column_format='l|ccc|')


## Robustness

Different parent and child ages.

Reorganize variables

In [90]:
# No age fixed effects necessary
new_names_var.index('age 35-40')
new_names_var2 = new_names_var.copy()
new_names_var2.pop(5)
new_names_var2.pop(5)

# For splitting the tables
cut1 = (new_names_var2.index('occ group 5')*2)+2
cut2 = (new_names_var2.index('occ group 2 x female')*2)
cut3 = (new_names_var2.index('occ group 5 x female')*2)+2
cut4 = len(new_names_var2)*2

In [93]:
path = './tables/'
omits = ['sexo1#c.ttrend','sexo1#c.ttrend2',"covid#sexo1","sexo1#occgroup","covid#sexo1#occgroup","sexo1#c.urate","occgroup","covid#occgroup"]


All_results= []

all_names = ['5','10','15']
noobs, r2ss = [], []
formal_names = [ "Children younger than 5","Children younger than 10","Children younger than 15"]

for age in all_names:
    name = "sqtreg_table_advanced_simple_age3035__{}".format(age)
    N, r2s = file_split('{}.log'.format(name),path,startwords=['q25','q50','q75'],endword="_cons",
               header="ten_y | Coefficient std t p_stat lower_95 upper_95",Noobs=True)
    noobs.append(N)
    r2ss.append(r2s)

    tab_moms_25 = read_table(path+'{}_1.log'.format(name),
                             keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)
    tab_moms_50 = read_table(path+'{}_2.log'.format(name),
                             keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)
    tab_moms_75 = read_table(path+'{}_3.log'.format(name),
                             keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)
    
    All_results.append([tab_moms_25,tab_moms_50,tab_moms_75])


res_names = ['25th pctl','50th pctl','75th pctl']
reorder_cols = new_names_var2
all_tabs = []

for j in range(len(all_names)):
    all_results = All_results[j]
    for i in range(len(all_results)):
        all_results[i] = rename_results(all_results[i].T,all_results[i].index,new_names_var2,new_names_var2)
    table = pd.DataFrame(pass_regression_to_latex_column(all_results[0].T),columns=[res_names[0],])
    for ir,res in enumerate(all_results[1:]):
        col = pass_regression_to_latex_column(res.T)
        table[res_names[ir+1]] = col.values


    for i,reg in enumerate(res_names):
        for var in reorder_cols:
            if all_results[i].loc['p_stat'][var] < 0.1 and all_results[i].loc['p_stat'][var] > 0.05:
                table[reg].loc[var] = r"${}^*$".format(table[reg].loc[var])
            elif all_results[i].loc['p_stat'][var] < 0.05 and all_results[i].loc['p_stat'][var] > 0.01:
                table[reg].loc[var] = r"${}^{{**}}$".format(table[reg].loc[var])
            elif all_results[i].loc['p_stat'][var] < 0.01:
                table[reg].loc[var] = r"${}^{{***}}$".format(table[reg].loc[var])
            else:
                table[reg].loc[var] = r"${}$".format(table[reg].loc[var])

    N_row = pd.DataFrame([N]*3, index=res_names, columns=['N',]).T
    R2_row = pd.DataFrame(r2s, index=res_names, columns=[r'Pseudo $R^2$',]).T
    table = pd.concat([table,N_row,R2_row])
    
    all_tabs.append(table)

Big_tab = pd.concat(all_tabs,axis=1,keys=(formal_names))
Big_tab_1 = pd.concat((Big_tab.iloc[:cut1,:],Big_tab.iloc[cut2:cut3,:],Big_tab.iloc[cut4-2:,:]))
Big_tab_2 = pd.concat((Big_tab.iloc[cut1:cut2,:],Big_tab.iloc[cut3:cut4-2,:],Big_tab.iloc[cut4:,:]))

Big_tab_1.to_latex('./tables/sqtreg_advanced_age3035_10_agefix_part1.tex',escape=False,column_format='l|ccc|ccc|ccc|')
Big_tab_2.to_latex('./tables/sqtreg_advanced_age3035_10_agefix_part2.tex',escape=False,column_format='l|ccc|ccc|ccc|')
print(Big_tab)

                                       Children younger than 5  \
                                                     25th pctl   
$t$                                             $-0.008^{***}$   
                                                      (0.0012)   
$t$ x female                                     $0.005^{***}$   
                                                      (0.0018)   
unemployment rate x male                         $0.034^{***}$   
                                                      (0.0031)   
unemployment rate x female                       $0.062^{***}$   
                                                      (0.0037)   
female                                           $-0.228^{**}$   
                                                      (0.1094)   
occ group 2                                     $-0.665^{***}$   
                                                      (0.0744)   
occ group 3                                     $-1.401^{***}$   
          

<ipython-input-93-a60f244af3b6>:63: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Big_tab_1.to_latex('./tables/sqtreg_advanced_age3035_10_agefix_part1.tex',escape=False,column_format='l|ccc|ccc|ccc|')
<ipython-input-93-a60f244af3b6>:64: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Big_tab_2.to_latex('./tables/sqtreg_advanced_age3035_10_agefix_part2.tex',escape=False,column_format='l|ccc|ccc|ccc|')


In [94]:
path = './tables/'
omits = ['sexo1#c.ttrend','sexo1#c.ttrend2',"covid#sexo1","sexo1#occgroup","covid#sexo1#occgroup","sexo1#c.urate","occgroup","covid#occgroup"]


All_results= []

all_names = ['5','10','15']
noobs, r2ss = [], []
formal_names = [ "Children younger than 5","Children younger than 10","Children younger than 15"]

for age in all_names:
    name = "sqtreg_table_advanced_simple_age3540__{}".format(age)
    N, r2s = file_split('{}.log'.format(name),path,startwords=['q25','q50','q75'],endword="_cons",
               header="ten_y | Coefficient std t p_stat lower_95 upper_95",Noobs=True)
    noobs.append(N)
    r2ss.append(r2s)

    tab_moms_25 = read_table(path+'{}_1.log'.format(name),
                             keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)
    tab_moms_50 = read_table(path+'{}_2.log'.format(name),
                             keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)
    tab_moms_75 = read_table(path+'{}_3.log'.format(name),
                             keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)
    
    All_results.append([tab_moms_25,tab_moms_50,tab_moms_75])


res_names = ['25th pctl','50th pctl','75th pctl']
reorder_cols = new_names_var2
all_tabs = []

for j in range(len(all_names)):
    all_results = All_results[j]
    for i in range(len(all_results)):
        all_results[i] = rename_results(all_results[i].T,all_results[i].index,new_names_var2,new_names_var2)
    table = pd.DataFrame(pass_regression_to_latex_column(all_results[0].T),columns=[res_names[0],])
    for ir,res in enumerate(all_results[1:]):
        col = pass_regression_to_latex_column(res.T)
        table[res_names[ir+1]] = col.values


    for i,reg in enumerate(res_names):
        for var in reorder_cols:
            if all_results[i].loc['p_stat'][var] < 0.1 and all_results[i].loc['p_stat'][var] > 0.05:
                table[reg].loc[var] = r"${}^*$".format(table[reg].loc[var])
            elif all_results[i].loc['p_stat'][var] < 0.05 and all_results[i].loc['p_stat'][var] > 0.01:
                table[reg].loc[var] = r"${}^{{**}}$".format(table[reg].loc[var])
            elif all_results[i].loc['p_stat'][var] < 0.01:
                table[reg].loc[var] = r"${}^{{***}}$".format(table[reg].loc[var])
            else:
                table[reg].loc[var] = r"${}$".format(table[reg].loc[var])

    N_row = pd.DataFrame([N]*3, index=res_names, columns=['N',]).T
    R2_row = pd.DataFrame(r2s, index=res_names, columns=[r'Pseudo $R^2$',]).T
    table = pd.concat([table,N_row,R2_row])
    
    all_tabs.append(table)

Big_tab = pd.concat(all_tabs,axis=1,keys=(formal_names))
Big_tab_1 = pd.concat((Big_tab.iloc[:cut1,:],Big_tab.iloc[cut2:cut3,:],Big_tab.iloc[cut4-2:,:]))
Big_tab_2 = pd.concat((Big_tab.iloc[cut1:cut2,:],Big_tab.iloc[cut3:cut4-2,:],Big_tab.iloc[cut4:,:]))

Big_tab_1.to_latex('./tables/sqtreg_advanced_age3540_part1.tex',escape=False,column_format='l|ccc|ccc|ccc|')
Big_tab_2.to_latex('./tables/sqtreg_advanced_age3540_part2.tex',escape=False,column_format='l|ccc|ccc|ccc|')
print(Big_tab)

                                       Children younger than 5  \
                                                     25th pctl   
$t$                                             $-0.013^{***}$   
                                                      (0.0013)   
$t$ x female                                     $0.019^{***}$   
                                                      (0.0019)   
unemployment rate x male                         $0.035^{***}$   
                                                      (0.0037)   
unemployment rate x female                       $0.046^{***}$   
                                                      (0.0039)   
female                                          $-0.622^{***}$   
                                                      (0.1248)   
occ group 2                                     $-0.336^{***}$   
                                                      (0.1017)   
occ group 3                                     $-2.467^{***}$   
          

<ipython-input-94-151947d24784>:63: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Big_tab_1.to_latex('./tables/sqtreg_advanced_age3540_part1.tex',escape=False,column_format='l|ccc|ccc|ccc|')
<ipython-input-94-151947d24784>:64: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Big_tab_2.to_latex('./tables/sqtreg_advanced_age3540_part2.tex',escape=False,column_format='l|ccc|ccc|ccc|')


Children younger than 5  \
                                                     25th pctl   
$\delta_{2020}$ x occ group 1                          $0.297$   
                                                      (0.3415)   
$\delta_{2020}$ x occ group 2                    $0.651^{***}$   
                                                      (0.2077)   
$\delta_{2020}$ x occ group 3                    $0.565^{***}$   
                                                      (0.1602)   
$\delta_{2020}$ x occ group 4                     $0.231^{**}$   
                                                      (0.1092)   
$\delta_{2020}$ x occ group 5                         $-0.125$   
                                                      (0.2082)   
$\delta_{2021}$ x occ group 1                         $-0.314$   
                                                      (0.4545)   
$\delta_{2021}$ x occ group 2                        $0.757^*$   
                                                      (0.4254)   
$\delta_{2021}$ x occ group 3                    $0.857^{***}$   
                                                      (0.2189)   
$\delta_{2021}$ x occ group 4                    $0.410^{***}$   
                                                      (0.0821)   
$\delta_{2021}$ x occ group 5                     $0.667^{**}$   
                                                      (0.2679)   
$\delta_{2022}$ x occ group 1                         $-0.355$   
                                                      (0.3600)   
$\delta_{2022}$ x occ group 2                          $0.082$   
                                                      (0.3076)   
$\delta_{2022}$ x occ group 3                        $0.235^*$   
                                                      (0.1357)   
$\delta_{2022}$ x occ group 4                    $0.436^{***}$   
                                                      (0.1226)   
$\delta_{2022}$ x occ group 5                         $-0.118$   
                                                      (0.1091)   
$\delta_{2020}$ x occ group 1 x female          $-0.921^{***}$   
                                                      (0.2859)   
$\delta_{2020}$ x occ group 2 x female                $-0.498$   
                                                      (0.3540)   
$\delta_{2020}$ x occ group 3 x female              $-0.468^*$   
                                                      (0.2531)   
$\delta_{2020}$ x occ group 4 x female              $-0.618^*$   
                                                      (0.3246)   
$\delta_{2020}$ x occ group 5 x female                $-0.095$   
                                                      (0.1110)   
$\delta_{2021}$ x occ group 1 x female           $-0.654^{**}$   
                                                      (0.3078)   
$\delta_{2021}$ x occ group 2 x female                $-0.139$   
                                                      (0.5187)   
$\delta_{2021}$ x occ group 3 x female                $-0.563$   
                                                      (0.4773)   
$\delta_{2021}$ x occ group 4 x female          $-1.162^{***}$   
                                                      (0.3257)   
$\delta_{2021}$ x occ group 5 x female          $-0.473^{***}$   
                                                      (0.1086)   
$\delta_{2022}$ x occ group 1 x female          $-0.903^{***}$   
                                                      (0.3416)   
$\delta_{2022}$ x occ group 2 x female                $-0.615$   
                                                      (0.4019)   
$\delta_{2022}$ x occ group 3 x female                 $0.167$   
                                                      (0.2626)   
$\delta_{2022}$ x occ group 4 x female                $-0.474$   
                                                      (0.3058)   
$\delta_{2022}$ x occ group 5 x female           $-0.278^{**}$   
             

In [ ]:
all_tabs = []

all_names = ['5','10','15']
formal_names = [ "Children younger than 5","Children younger than 10","Children younger than 15"]

for name in all_names:
    tab = Table.read('./regtabs/sqtreg_advanced_age3540_{}_controls.tex'.format(name)).to_pandas()
    tab = tab.set_index('col0')
    tab.rename(columns=dict(zip(tab.keys(),[i.replace('Parents,',"") for i in tab.keys()])),inplace=True)
    all_tabs.append(tab)
    
Btab = pd.concat(all_tabs,axis=1,keys=(formal_names))

Btab.index = [str(i).replace("nan","") for i in Btab.index]
# Btab.to_latex('./tables/sqtreg_advanced_age3540_all_controls.tex',escape=False,column_format='l|ccc|ccc|ccc',multicolumn_format='c')
Btab

In [ ]:
path = './tables/'
omits = ['sexo1#c.ttrend','sexo1#c.ttrend2',"covid#sexo1","sexo1#occgroup","covid#sexo1#occgroup","sexo1#c.urate","occgroup","covid#occgroup"]

name = "sqtreg_table_advanced_simple_age3040__10_agefix"
N, r2s = file_split('{}.log'.format(name),path,startwords=['q25','q50','q75'],endword="_cons",
           header="ten_y | Coefficient std t p_stat lower_95 upper_95",Noobs=True)

tab_moms_25_0k = read_table(path+'{}_1.log'.format(name),
                         keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)
tab_moms_50_0k = read_table(path+'{}_2.log'.format(name),
                         keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)
tab_moms_75_0k = read_table(path+'{}_3.log'.format(name),
                         keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)


In [7]:
res_names = ['Parents, 25th pctl','No children, 25th pctl','Parents, 50th pctl',
            'No children, 50th pctl','Parents, 75th pctl','No Children, 75th pctl']

all_results = [tab_moms_25,tab_moms_25_0k,tab_moms_50,
               tab_moms_50_0k,tab_moms_75,tab_moms_75_0k]


for i in range(len(all_results)):
    all_results[i] = rename_results(all_results[i].T)
table = pd.DataFrame(pass_regression_to_latex_column(all_results[0].T),columns=[res_names[0],])
for ir,res in enumerate(all_results[1:]):
    col = pass_regression_to_latex_column(res.T)
    table[res_names[ir+1]] = col.values
    
    
for i,reg in enumerate(res_names):
    for var in reorder_cols:
        if all_results[i].loc['p_stat'][var] < 0.1 and all_results[i].loc['p_stat'][var] > 0.05:
            table[reg].loc[var] = r"${}^*$".format(table[reg].loc[var])
        elif all_results[i].loc['p_stat'][var] < 0.05 and all_results[i].loc['p_stat'][var] > 0.01:
            table[reg].loc[var] = r"${}^{{**}}$".format(table[reg].loc[var])
        elif all_results[i].loc['p_stat'][var] < 0.01:
            table[reg].loc[var] = r"${}^{{***}}$".format(table[reg].loc[var])
        else:
            table[reg].loc[var] = r"${}$".format(table[reg].loc[var])
            
N_row = pd.DataFrame(np.tile([N, N_0k],3), index=res_names, columns=['N',]).T
R2_row = pd.DataFrame(np.vstack((r2s,r2s_0k)).T.ravel(), index=res_names, columns=[r'Pseudo $R^2$',]).T
table = pd.concat([table,N_row,R2_row])

# print(table.loc[[r'$\delta_{2020}$','$\delta_{2021}$',r'$\delta_{2020}$ x female',
#                      r'$\delta_{2021}$ x female']].to_latex(escape=False))
table

,"Parents, 25th pctl","No children, 25th pctl","Parents, 50th pctl","No children, 50th pctl","Parents, 75th pctl","No Children, 75th pctl"
$\beta_0$,$0.866^{***}$,$1.226^{***}$,$3.335^{***}$,$3.594^{***}$,$7.678^{***}$,$7.030^{***}$
,(0.0600),(0.0512),(0.0850),(0.0849),(0.1066),(0.1328)
female,$-0.820^{***}$,$-0.531^{***}$,$-1.379^{***}$,$-0.999^{***}$,$-1.297^{***}$,$-1.402^{***}$
,(0.0748),(0.0799),(0.1182),(0.1225),(0.1232),(0.1737)
partner tenure (years),$0.102^{***}$,$0.070^{***}$,$0.205^{***}$,$0.144^{***}$,$0.169^{***}$,$0.160^{***}$
,(0.0025),(0.0043),(0.0041),(0.0042),(0.0053),(0.0063)
$t$,$0.054^{***}$,$0.039^{***}$,$0.075^{***}$,$0.051^{***}$,$0.020^{**}$,$0.020^*$
,(0.0063),(0.0059),(0.0063),(0.0066),(0.0093),(0.0104)
$t^{2}$,$-0.001^{***}$,$-0.001^{***}$,$-0.001^{***}$,$-0.001^{***}$,$0.000$,$-0.000$
,(0.0001),(0.0001),(0.0001),(0.0001),(0.0002),(0.0002)


In [404]:
res_names = ['Parents, 25th pctl','No children, 25th pctl','Parents, 50th pctl',
            'No children, 50th pctl','Parents, 75th pctl','No Children, 75th pctl']

all_results = [tab_moms_25,tab_moms_25_0k,tab_moms_50,
               tab_moms_50_0k,tab_moms_75,tab_moms_75_0k]


for i in range(len(all_results)):
    all_results[i] = rename_results(all_results[i].T)
table = pd.DataFrame(pass_regression_to_latex_column(all_results[0].T),columns=[res_names[0],])
for ir,res in enumerate(all_results[1:]):
    col = pass_regression_to_latex_column(res.T)
    table[res_names[ir+1]] = col.values
    
    
for i,reg in enumerate(res_names):
    for var in reorder_cols:
        if all_results[i].loc['p_stat'][var] < 0.1 and all_results[i].loc['p_stat'][var] > 0.05:
            table[reg].loc[var] = r"${}^*$".format(table[reg].loc[var])
        elif all_results[i].loc['p_stat'][var] < 0.05 and all_results[i].loc['p_stat'][var] > 0.01:
            table[reg].loc[var] = r"${}^{{**}}$".format(table[reg].loc[var])
        elif all_results[i].loc['p_stat'][var] < 0.01:
            table[reg].loc[var] = r"${}^{{***}}$".format(table[reg].loc[var])
        else:
            table[reg].loc[var] = r"${}$".format(table[reg].loc[var])
            
N_row = pd.DataFrame(np.tile([N, N_0k],3), index=res_names, columns=['N',]).T
R2_row = pd.DataFrame(np.vstack((r2s,r2s_0k)).T.ravel(), index=res_names, columns=[r'Pseudo $R^2$',]).T
table = pd.concat([table,N_row,R2_row])

# print(table.loc[[r'$\delta_{2020}$','$\delta_{2021}$',r'$\delta_{2020}$ x female',
#                      r'$\delta_{2021}$ x female']].to_latex(escape=False))
#print(table.to_latex(escape=False))

\begin{tabular}{lllllll}
\toprule
{} & Parents, 25th pctl & No children, 25th pctl & Parents, 50th pctl & No children, 50th pctl & Parents, 75th pctl & No Children, 75th pctl \\
\midrule
$\beta_0$                &      $0.858^{***}$ &          $1.221^{***}$ &      $3.323^{***}$ &          $3.571^{***}$ &      $7.722^{***}$ &          $7.029^{***}$ \\
                         &           (0.0578) &               (0.0550) &           (0.0852) &               (0.0516) &           (0.1003) &               (0.1210) \\
female                   &     $-0.829^{***}$ &         $-0.528^{***}$ &     $-1.379^{***}$ &         $-0.982^{***}$ &     $-1.298^{***}$ &         $-1.381^{***}$ \\
                         &           (0.0870) &               (0.0577) &           (0.1384) &               (0.0997) &           (0.1757) &               (0.1369) \\
partner tenure (years)   &      $0.103^{***}$ &          $0.070^{***}$ &      $0.209^{***}$ &          $0.144^{***}$ &      $0.170^{***}$ &          

In [376]:
table.head()

,"Parents, 25th pctl","No children, 25th pctl","Parents, 50th pctl","No children, 50th pctl","Parents, 75th pctl","No Children, 75th pctl"
$\beta_0$,$0.858^{***}$,$1.221^{***}$,$3.323^{***}$,$3.571^{***}$,$7.722^{***}$,$7.029^{***}$
,(0.0578),(0.0550),(0.0852),(0.0516),(0.1003),(0.1210)
female,$-0.829^{***}$,$-0.528^{***}$,$-1.379^{***}$,$-0.982^{***}$,$-1.298^{***}$,$-1.381^{***}$
,(0.0870),(0.0577),(0.1384),(0.0997),(0.1757),(0.1369)
partner tenure (years),$0.103^{***}$,$0.070^{***}$,$0.209^{***}$,$0.144^{***}$,$0.170^{***}$,$0.161^{***}$


In [8]:
path = './results/'
omits = ['sexo1#c.ttrend','sexo1#c.ttrend2',"covid#sexo1"]
# int_subs ={}
# for t in time_periods:
#     int_subs[t] = 'ten_'+t[1:]
name = "sqtreg_3035_hemp_time_int_diff"
N, r2s = file_split('{}.log'.format(name),path,startwords=['q25','q50','q75'],endword="_cons",
           header="ten_y | Coefficient std t p_stat lower_95 upper_95",Noobs=True)

tab_moms_25 = read_table(path+'{}_1.log'.format(name),
                         keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)
tab_moms_50 = read_table(path+'{}_2.log'.format(name),
                         keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)
tab_moms_75 = read_table(path+'{}_3.log'.format(name),
                         keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)

name = "sqtreg_3035_hemp_time_int_diff_0k"
N_0k, r2s_0k = file_split('{}.log'.format(name),path,startwords=['q25','q50','q75'],endword="_cons",
           header="ten_y | Coefficient std t p_stat lower_95 upper_95",Noobs=True)

tab_moms_25_0k = read_table(path+'{}_1.log'.format(name),
                         keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)
tab_moms_50_0k = read_table(path+'{}_2.log'.format(name),
                         keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)
tab_moms_75_0k = read_table(path+'{}_3.log'.format(name),
                         keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)

In [9]:
old_names_var = ['other_ten_y', 'ttrend', 'ttrend2', 'sexo1#c.ttrend_1',
       'sexo1#c.ttrend2_1', 'covid_1', 'covid_2', '1.sexo1', 'covid#sexo1_1',
       'covid#sexo1_2', 'part_time#sexo1_0', 'part_time#sexo1_1',
       'college#sexo1_0', 'college#sexo1_1', 'less_hs#sexo1_0',
       'less_hs#sexo1_1', 'other_se#sexo1_0', 'other_se#sexo1_1',
       'other_college#sexo1_0', 'other_college#sexo1_1', 
        'other_less_hs#sexo1_0', 'other_less_hs#sexo1_1','_cons']
new_names_var = [r'partner tenure (years)',r'$t$',r'$t^{2}$',r'$t$ x female',r'$t^2$ x female',
                     r'$\delta_{2020}$','$\delta_{2021}$','female',r'$\delta_{2020}$ x female',
                     r'$\delta_{2021}$ x female','part-time','part-time x female',
       'college', 'college x female', 'less than HS',
       'less_than HS x female', 'partner is SE', 'partner is SE x female',
       'partner college', 'partner college x female',
       'partner less than HS', 'partner less than HS x female',r'$\beta_0$']
reorder_cols = [r'$\beta_0$','female',r'partner tenure (years)',r'$t$',r'$t^{2}$',r'$t$ x female',r'$t^2$ x female',
                r'$\delta_{2020}$',r'$\delta_{2021}$',r'$\delta_{2020}$ x female',r'$\delta_{2021}$ x female',
                'part-time','part-time x female','college', 'college x female', 'less than HS',
                'less_than HS x female', 'partner is SE', 'partner is SE x female','partner college', 'partner college x female',
       'partner less than HS', 'partner less than HS x female']

def rename_results(df,old_names=old_names_var,new_names=new_names_var,order=reorder_cols):
    
    df.rename(columns=dict(zip(old_names,new_names)),inplace='True')
    df = df.reindex(columns=order)
    return df

table = rename_results(tab_moms_25.T)

In [10]:
res_names = ['Parents, 25th pctl','No children, 25th pctl','Parents, 50th pctl',
            'No children, 50th pctl','Parents, 75th pctl','No Children, 75th pctl']

all_results = [tab_moms_25,tab_moms_25_0k,tab_moms_50,
               tab_moms_50_0k,tab_moms_75,tab_moms_75_0k]


for i in range(len(all_results)):
    all_results[i] = rename_results(all_results[i].T)
table = pd.DataFrame(pass_regression_to_latex_column(all_results[0].T),columns=[res_names[0],])
for ir,res in enumerate(all_results[1:]):
    col = pass_regression_to_latex_column(res.T)
    table[res_names[ir+1]] = col.values
    
    
for i,reg in enumerate(res_names):
    for var in reorder_cols:
        if all_results[i].loc['p_stat'][var] <= 0.1 and all_results[i].loc['p_stat'][var] > 0.05:
            table[reg].loc[var] = r"${}^*$".format(table[reg].loc[var])
        elif all_results[i].loc['p_stat'][var] <= 0.05 and all_results[i].loc['p_stat'][var] > 0.01:
            table[reg].loc[var] = r"${}^{{**}}$".format(table[reg].loc[var])
        elif all_results[i].loc['p_stat'][var] <= 0.01:
            table[reg].loc[var] = r"${}^{{***}}$".format(table[reg].loc[var])
        else:
            table[reg].loc[var] = r"${}$".format(table[reg].loc[var])
            
N_row = pd.DataFrame(np.tile([N, N_0k],3), index=res_names, columns=['N',]).T
R2_row = pd.DataFrame(np.vstack((r2s,r2s_0k)).T.ravel(), index=res_names, columns=[r'Pseudo $R^2$',]).T
table = pd.concat([table,N_row,R2_row])

#print(table.to_latex(escape=False))
table

,"Parents, 25th pctl","No children, 25th pctl","Parents, 50th pctl","No children, 50th pctl","Parents, 75th pctl","No Children, 75th pctl"
$\beta_0$,$0.743^{***}$,$0.970^{***}$,$3.124^{***}$,$3.601^{***}$,$8.390^{***}$,$7.899^{***}$
,(0.0678),(0.0646),(0.1132),(0.0961),(0.0952),(0.1096)
female,$-0.598^{***}$,$-0.137$,$-0.965^{***}$,$-0.862^{***}$,$-1.611^{***}$,$-1.100^{***}$
,(0.0711),(0.1004),(0.1626),(0.0969),(0.1392),(0.1456)
partner tenure (years),$0.079^{***}$,$0.062^{***}$,$0.176^{***}$,$0.136^{***}$,$0.166^{***}$,$0.150^{***}$
,(0.0040),(0.0034),(0.0072),(0.0056),(0.0045),(0.0068)
$t$,$0.045^{***}$,$0.038^{***}$,$0.074^{***}$,$0.045^{***}$,$0.017^{**}$,$0.028^{**}$
,(0.0054),(0.0047),(0.0080),(0.0088),(0.0077),(0.0114)
$t^{2}$,$-0.001^{***}$,$-0.001^{***}$,$-0.001^{***}$,$-0.001^{***}$,$0.000$,$-0.000$
,(0.0001),(0.0001),(0.0001),(0.0001),(0.0001),(0.0002)


In [400]:
res_names = ['Parents, 25th pctl','No children, 25th pctl','Parents, 50th pctl',
            'No children, 50th pctl','Parents, 75th pctl','No Children, 75th pctl']

all_results = [tab_moms_25,tab_moms_25_0k,tab_moms_50,
               tab_moms_50_0k,tab_moms_75,tab_moms_75_0k]


for i in range(len(all_results)):
    all_results[i] = rename_results(all_results[i].T)
table = pd.DataFrame(pass_regression_to_latex_column(all_results[0].T),columns=[res_names[0],])
for ir,res in enumerate(all_results[1:]):
    col = pass_regression_to_latex_column(res.T)
    table[res_names[ir+1]] = col.values
    
    
for i,reg in enumerate(res_names):
    for var in reorder_cols:
        if all_results[i].loc['p_stat'][var] <= 0.1 and all_results[i].loc['p_stat'][var] > 0.05:
            table[reg].loc[var] = r"${}^*$".format(table[reg].loc[var])
        elif all_results[i].loc['p_stat'][var] <= 0.05 and all_results[i].loc['p_stat'][var] > 0.01:
            table[reg].loc[var] = r"${}^{{**}}$".format(table[reg].loc[var])
        elif all_results[i].loc['p_stat'][var] <= 0.01:
            table[reg].loc[var] = r"${}^{{***}}$".format(table[reg].loc[var])
        else:
            table[reg].loc[var] = r"${}$".format(table[reg].loc[var])
            
N_row = pd.DataFrame(np.tile([N, N_0k],3), index=res_names, columns=['N',]).T
R2_row = pd.DataFrame(np.vstack((r2s,r2s_0k)).T.ravel(), index=res_names, columns=[r'Pseudo $R^2$',]).T
table = pd.concat([table,N_row,R2_row])

print(table.to_latex(escape=False))

\begin{tabular}{lllllll}
\toprule
{} & Parents, 25th pctl & No children, 25th pctl & Parents, 50th pctl & No children, 50th pctl & Parents, 75th pctl & No Children, 75th pctl \\
\midrule
$\beta_0$                     &      $0.744^{***}$ &          $0.961^{***}$ &      $3.116^{***}$ &          $3.540^{***}$ &      $8.427^{***}$ &          $7.939^{***}$ \\
                              &           (0.0845) &               (0.0521) &           (0.1081) &               (0.0939) &           (0.1157) &               (0.1480) \\
female                        &     $-0.630^{***}$ &               $-0.128$ &     $-0.974^{***}$ &         $-0.803^{***}$ &     $-1.597^{***}$ &         $-1.073^{***}$ \\
                              &           (0.0921) &               (0.0804) &           (0.1280) &               (0.1079) &           (0.1574) &               (0.1736) \\
partner tenure (years)        &      $0.080^{***}$ &          $0.061^{***}$ &      $0.180^{***}$ &          $0.136^{***}$ &      

In [11]:
path = './results/'
omits = ['sexo1#c.ttrend','sexo1#c.ttrend2',"covid#sexo1"]
tenure_bins = np.arange(1,4)
tables_cond_ten = {}
tables_cond_ten_0k = {}
Ns, r2s, Ns_0k, r2s_0k = {}, {}, {}, {}
# for t in time_periods:
#     int_subs[t] = 'ten_'+t[1:]
for t in tenure_bins:
    name = "sqtreg_3035_hemp_time_int_diff_t{}".format(t)
    Ns[t], r2s[t] = file_split('{}.log'.format(name),path,startwords=['q25','q50','q75'],endword="_cons",
               header="ten_y | Coefficient std t p_stat lower_95 upper_95",Noobs=True)
    dict_i = {}
    for i,q in enumerate([25,50,75]):
        tab_i = read_table(path+'{}_{}.log'.format(name,i+1),
                             keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)   
        dict_i[q] = tab_i
    tables_cond_ten[t] = dict_i.copy()
    
    dict_i = {}
    name = "sqtreg_3035_hemp_time_int_diff_t{}_0k".format(t)
    Ns_0k[t], r2s_0k[t] = file_split('{}.log'.format(name),path,startwords=['q25','q50','q75'],endword="_cons",
               header="ten_y | Coefficient std t p_stat lower_95 upper_95",Noobs=True)
    for i,q in enumerate([25,50,75]):
        tab_i_0k = read_table(path+'{}_{}.log'.format(name,i+1),
                             keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)    
        dict_i[q] = tab_i_0k
    tables_cond_ten_0k[t] = dict_i.copy()

FileNotFoundError: [Errno 2] No such file or directory: './results/sqtreg_3035_hemp_time_int_diff_t1.log'

In [406]:
old_names_var = ['ttrend', 'ttrend2', 'sexo1#c.ttrend_1',
       'sexo1#c.ttrend2_1', 'covid_1', 'covid_2', '1.sexo1', 'covid#sexo1_1',
       'covid#sexo1_2', 'part_time#sexo1_0', 'part_time#sexo1_1',
       'college#sexo1_0', 'college#sexo1_1', 'less_hs#sexo1_0',
       'less_hs#sexo1_1', 'other_se#sexo1_0', 'other_se#sexo1_1',
       'other_college#sexo1_0', 'other_college#sexo1_1', 
        'other_less_hs#sexo1_0', 'other_less_hs#sexo1_1','_cons']
new_names_var = [r'$t$',r'$t^{2}$',r'$t$ x female',r'$t^2$ x female',
                     r'$\delta_{2020}$','$\delta_{2021}$','female',r'$\delta_{2020}$ x female',
                     r'$\delta_{2021}$ x female','part-time','part-time x female',
       'college', 'college x female', 'less than HS',
       'less_than HS x female', 'partner is SE', 'partner is SE x female',
       'partner college', 'partner college x female',
       'partner less than HS', 'partner less than HS x female',r'$\beta_0$']
reorder_cols = [r'$\beta_0$','female',r'$t$',r'$t^{2}$',r'$t$ x female',r'$t^2$ x female',
                r'$\delta_{2020}$',r'$\delta_{2021}$',r'$\delta_{2020}$ x female',r'$\delta_{2021}$ x female',
                'part-time','part-time x female','college', 'college x female', 'less than HS',
                'less_than HS x female', 'partner is SE', 'partner is SE x female','partner college', 'partner college x female',
       'partner less than HS', 'partner less than HS x female']

def rename_results(df,old_names=old_names_var,new_names=new_names_var,order=reorder_cols):    
    df.rename(columns=dict(zip(old_names,new_names)),inplace='True')
    df = df.reindex(columns=order)
    return df


In [411]:
res_names = ['Parents, 25th pctl','No children, 25th pctl','Parents, 50th pctl',
            'No children, 50th pctl','Parents, 75th pctl','No Children, 75th pctl']
for ten_level in range(1,4):
    all_results = []
    for i,q in enumerate([25,50,75]):
        all_results.append(tables_cond_ten[ten_level][q])
        all_results.append(tables_cond_ten_0k[ten_level][q])


    for i in range(len(all_results)):
        all_results[i] = rename_results(all_results[i].T)
    table = pd.DataFrame(pass_regression_to_latex_column(all_results[0].T),columns=[res_names[0],])
    for ir,res in enumerate(all_results[1:]):
        col = pass_regression_to_latex_column(res.T)
        table[res_names[ir+1]] = col.values


    for i,reg in enumerate(res_names):
        for var in reorder_cols:
            if all_results[i].loc['p_stat'][var] <= 0.1 and all_results[i].loc['p_stat'][var] > 0.05:
                table[reg].loc[var] = r"${}^*$".format(table[reg].loc[var])
            elif all_results[i].loc['p_stat'][var] <= 0.05 and all_results[i].loc['p_stat'][var] > 0.01:
                table[reg].loc[var] = r"${}^{{**}}$".format(table[reg].loc[var])
            elif all_results[i].loc['p_stat'][var] <= 0.01:
                table[reg].loc[var] = r"${}^{{***}}$".format(table[reg].loc[var])
            else:
                table[reg].loc[var] = r"${}$".format(table[reg].loc[var])
                
    N_row = pd.DataFrame(np.tile([Ns[ten_level], Ns_0k[ten_level]],3), index=res_names, columns=['N',]).T
    R2_row = pd.DataFrame(np.vstack((r2s[ten_level],r2s_0k[ten_level])).T.ravel(), index=res_names, columns=[r'Pseudo $R^2$',]).T
    table = pd.concat([table,N_row,R2_row])

    # print(table.to_latex(escape=False))
    print(table.loc[[r'$\delta_{2020}$','$\delta_{2021}$',r'$\delta_{2020}$ x female',
                         r'$\delta_{2021}$ x female','N',r'Pseudo $R^2$']].to_latex(escape=False))

\begin{tabular}{lllllll}
\toprule
{} & Parents, 25th pctl & No children, 25th pctl & Parents, 50th pctl & No children, 50th pctl & Parents, 75th pctl & No Children, 75th pctl \\
\midrule
$\delta_{2020}$          &            $0.111$ &                $0.065$ &           $-0.174$ &               $-0.415$ &            $0.036$ &         $-1.850^{***}$ \\
$\delta_{2021}$          &            $0.046$ &                $0.078$ &            $0.536$ &               $-0.281$ &            $0.359$ &               $-1.303$ \\
$\delta_{2020}$ x female &           $-0.044$ &               $-0.042$ &           $-0.185$ &               $-0.069$ &           $-1.382$ &               $-0.181$ \\
$\delta_{2021}$ x female &            $0.347$ &                $0.174$ &            $0.277$ &                $0.668$ &            $0.729$ &                $0.311$ \\
N                        &             24,058 &                 17,955 &             24,058 &                 17,955 &             24,058 &          

In [412]:
path = './results/'
omits = ['sexo1#c.ttrend','sexo1#c.ttrend2',"covid#sexo1"]
tenure_bins = np.arange(1,4)
tables_cond_ten = {}
tables_cond_ten_0k = {}
Ns, r2s, Ns_0k, r2s_0k = {}, {}, {}, {}

for t in tenure_bins:
    name = "sqtreg_3035_hemp_time_int_diff_simple_t{}".format(t)
    Ns[t], r2s[t] = file_split('{}.log'.format(name),path,startwords=['q25','q50','q75'],endword="_cons",
               header="ten_y | Coefficient std t p_stat lower_95 upper_95",Noobs=True)
    dict_i = {}
    for i,q in enumerate([25,50,75]):
        tab_i = read_table(path+'{}_{}.log'.format(name,i+1),
                             keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)   
        dict_i[q] = tab_i
    tables_cond_ten[t] = dict_i.copy()
    
    dict_i_0k = {}
    name = "sqtreg_3035_hemp_time_int_diff_simple_t{}_0k".format(t)
    Ns_0k[t], r2s_0k[t] = file_split('{}.log'.format(name),path,startwords=['q25','q50','q75'],endword="_cons",
               header="ten_y | Coefficient std t p_stat lower_95 upper_95",Noobs=True)
    for i,q in enumerate([25,50,75]):
        tab_i_0k = read_table(path+'{}_{}.log'.format(name,i+1),
                             keyword='ten_y',rescale=1,ncols=6,start=2,omit_list=omits,index_to_int=False)    
        dict_i_0k[q] = tab_i_0k
    tables_cond_ten_0k[t] = dict_i_0k.copy()

In [413]:
old_names_var = ['ttrend', 'ttrend2', 'sexo1#c.ttrend_1',
       'sexo1#c.ttrend2_1', 'covid_1', 'covid_2', '1.sexo1', 'covid#sexo1_1',
       'covid#sexo1_2', 'part_time#sexo1_0', 'part_time#sexo1_1',
       'college#sexo1_0', 'college#sexo1_1', 'less_hs#sexo1_0',
       'less_hs#sexo1_1', 'other_se#sexo1_0', 'other_se#sexo1_1',
       'other_college#sexo1_0', 'other_college#sexo1_1', 
        'other_less_hs#sexo1_0', 'other_less_hs#sexo1_1','_cons']
new_names_var = [r'$t$',r'$t^{2}$',r'$t$ x female',r'$t^2$ x female',
                     r'$\delta_{2020}$','$\delta_{2021}$','female',r'$\delta_{2020}$ x female',
                     r'$\delta_{2021}$ x female','part-time','part-time x female',
       'college', 'college x female', 'less than HS',
       'less_than HS x female', 'partner is SE', 'partner is SE x female',
       'partner college', 'partner college x female',
       'partner less than HS', 'partner less than HS x female',r'$\beta_0$']
reorder_cols = [r'$\beta_0$','female',r'$t$',r'$t^{2}$',r'$t$ x female',r'$t^2$ x female',
                r'$\delta_{2020}$',r'$\delta_{2021}$',r'$\delta_{2020}$ x female',r'$\delta_{2021}$ x female',
                'part-time','part-time x female','college', 'college x female', 'less than HS',
                'less_than HS x female', 'partner is SE', 'partner is SE x female','partner college', 'partner college x female',
       'partner less than HS', 'partner less than HS x female']

def rename_results(df,old_names=old_names_var,new_names=new_names_var,order=reorder_cols):    
    df.rename(columns=dict(zip(old_names,new_names)),inplace='True')
    df = df.reindex(columns=order)
    return df


In [415]:
res_names = ['Parents, 25th pctl','No children, 25th pctl','Parents, 50th pctl',
            'No children, 50th pctl','Parents, 75th pctl','No Children, 75th pctl']
for ten_level in range(1,4):
    all_results = []
    for i,q in enumerate([25,50,75]):
        all_results.append(tables_cond_ten[ten_level][q])
        all_results.append(tables_cond_ten_0k[ten_level][q])


    for i in range(len(all_results)):
        all_results[i] = rename_results(all_results[i].T)
    table = pd.DataFrame(pass_regression_to_latex_column(all_results[0].T),columns=[res_names[0],])
    for ir,res in enumerate(all_results[1:]):
        col = pass_regression_to_latex_column(res.T)
        table[res_names[ir+1]] = col.values


    for i,reg in enumerate(res_names):
        for var in reorder_cols:
            if all_results[i].loc['p_stat'][var] <= 0.1 and all_results[i].loc['p_stat'][var] > 0.05:
                table[reg].loc[var] = r"${}^*$".format(table[reg].loc[var])
            elif all_results[i].loc['p_stat'][var] <= 0.05 and all_results[i].loc['p_stat'][var] > 0.01:
                table[reg].loc[var] = r"${}^{{**}}$".format(table[reg].loc[var])
            elif all_results[i].loc['p_stat'][var] <= 0.01:
                table[reg].loc[var] = r"${}^{{***}}$".format(table[reg].loc[var])
            else:
                table[reg].loc[var] = r"${}$".format(table[reg].loc[var])
                
    N_row = pd.DataFrame(np.tile([Ns[ten_level], Ns_0k[ten_level]],3), index=res_names, columns=['N',]).T
    R2_row = pd.DataFrame(np.vstack((r2s[ten_level],r2s_0k[ten_level])).T.ravel(), index=res_names, columns=[r'Pseudo $R^2$',]).T
    table = pd.concat([table,N_row,R2_row])

    # print(table.to_latex(escape=False))
    print(table.loc[[r'$\delta_{2020}$','$\delta_{2021}$',r'$\delta_{2020}$ x female',
                         r'$\delta_{2021}$ x female','N',r'Pseudo $R^2$']].to_latex(escape=False))

\begin{tabular}{lllllll}
\toprule
{} & Parents, 25th pctl & No children, 25th pctl & Parents, 50th pctl & No children, 50th pctl & Parents, 75th pctl & No Children, 75th pctl \\
\midrule
$\delta_{2020}$          &          $0.224^*$ &                $0.087$ &            $0.240$ &               $-0.016$ &            $0.046$ &         $-2.362^{***}$ \\
$\delta_{2021}$          &            $0.189$ &               $-0.155$ &            $0.669$ &                $0.027$ &            $0.386$ &          $-1.697^{**}$ \\
$\delta_{2020}$ x female &            $0.009$ &               $-0.026$ &            $0.031$ &               $-0.415$ &      $-1.799^{**}$ &                $0.283$ \\
$\delta_{2021}$ x female &            $0.289$ &          $0.499^{***}$ &            $0.921$ &                $0.532$ &           $-0.064$ &                $0.923$ \\
N                        &             24,058 &                 17,955 &             24,058 &                 17,955 &             24,058 &          

In [342]:
table.loc[[r'$\delta_{2020}$','$\delta_{2021}$',r'$\delta_{2020}$ x female',
                         r'$\delta_{2021}$ x female']]

,"Parents, 25th pctl","No children, 25th pctl","Parents, 50th pctl","No children, 50th pctl","Parents, 75th pctl","No Children, 75th pctl"
$\delta_{2020}$,$0.935^{***}$,$0.935^{***}$,$0.338$,$0.338$,$-0.388^*$,$-0.388^*$
$\delta_{2021}$,$1.199^{***}$,$1.199^{***}$,$0.704^{**}$,$0.704^{**}$,$-0.532^{**}$,$-0.532^{**}$
$\delta_{2020}$ x female,$-0.642^{**}$,$-0.642^{**}$,$-1.381^{***}$,$-1.381^{***}$,$0.025$,$0.025$
$\delta_{2021}$ x female,$-0.823^{***}$,$-0.823^{***}$,$-2.490^{***}$,$-2.490^{***}$,$-0.969^{***}$,$-0.969^{***}$


## Main Figure 5 tables

In [10]:
all_tabs = []

all_names = ['mothers','fathers']
formal_names = ['Women ', 'Men']

for name in all_names:
    tab = Table.read('./regtabs/tex/sqtreg_figure5_{}_age3040_10_agefix.tex'.format(name)).to_pandas()
    tab = tab.set_index('col0')
    tab.rename(columns=dict(zip(tab.keys(),[i.replace('Parents,',"") for i in tab.keys()])),inplace=True)
    all_tabs.append(tab)
    
Btab = pd.concat(all_tabs,axis=1,keys=(formal_names))

Btab.index = [str(i).replace("nan","") for i in Btab.index]

# Btab.to_latex('./tables/qtregs_3040_agefix.tex',escape=False)
print(Btab.to_latex())

\begin{tabular}{lllllll}
\toprule
{} & \multicolumn{3}{l}{Women} & \multicolumn{3}{l}{Men} \\
{} &       25th pctl &       50th pctl &       75th pctl &       25th pctl &       50th pctl &       75th pctl \\
\midrule
\$\textbackslash beta\_0\$                     &   \$0.640\textasciicircum \{***\}\$ &   \$3.000\textasciicircum \{***\}\$ &   \$6.403\textasciicircum \{***\}\$ &        \$-0.212\$ &   \$1.516\textasciicircum \{***\}\$ &   \$6.298\textasciicircum \{***\}\$ \\
                              &        (0.0652) &        (0.1387) &        (0.1970) &        (0.1490) &        (0.2298) &        (0.2888) \\
Tenure Partner                &   \$0.191\textasciicircum \{***\}\$ &   \$0.319\textasciicircum \{***\}\$ &   \$0.197\textasciicircum \{***\}\$ &   \$0.246\textasciicircum \{***\}\$ &   \$0.320\textasciicircum \{***\}\$ &   \$0.118\textasciicircum \{***\}\$ \\
                              &        (0.0056) &        (0.0076) &        (0.0089) &        (0.0132) &        (0.0173) &

<ipython-input-10-97fa7f3d56cd>:17: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(Btab.to_latex())


In [31]:
from astropy.table import Table

all_tabs = []

all_names = ['mothers0k','fathers0k']
formal_names = [ "Women ","Men "]

for name in all_names:
    tab = Table.read('./regtabs/tex/sqtreg_figure5_{}_age3040_10_agefix.tex'.format(name)).to_pandas()
    tab = tab.set_index('col0')
    tab.rename(columns=dict(zip(tab.keys(),[i.replace('Parents,',"") for i in tab.keys()])),inplace=True)
    all_tabs.append(tab)
    
Btab = pd.concat(all_tabs,axis=1,keys=(formal_names))

Btab.index = [str(i).replace("nan","") for i in Btab.index]

Btab.to_latex('./tables/qtregs_3040_agefix_0k.tex',escape=False)

C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_6180\905960972.py:18: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Btab.to_latex('./tables/qtregs_3040_agefix_0k.tex',escape=False)


In [9]:
IoI = []
std_idx = []
for i,idx in enumerate(tab.index):
    if str(idx)!="nan" and idx not in tab.index[-2:]:        
        IoI.append(idx)
        std_idx.append(i+1)
tab_dummies_std_f=tab.iloc[std_idx,:].copy()
tab_dummies_std_f['coefs'] = IoI
tab_dummies_std_f.set_index('coefs',inplace=True)
tab_dummies_f=tab.loc[IoI,:].copy()
tab_dummies_f

,"Parents, 25th pctl","Parents, 50th pctl","Parents, 75th pctl"
col0,,,
$\beta_0$,$-0.212$,$1.516^{***}$,$6.298^{***}$
Tenure Partner,$0.246^{***}$,$0.320^{***}$,$0.118^{***}$
Tenure \ Partner$^{2}$,$-0.007^{***}$,$-0.005^{***}$,$0.004^{***}$
Part Time,$-1.873^{***}$,$-3.817^{***}$,$-4.302^{***}$
College,$0.966^{***}$,$0.460^{***}$,$-1.225^{***}$
Less than High-school,$-0.653^{***}$,$-1.727^{***}$,$-2.587^{***}$
Partner age,$0.032^{***}$,$0.061^{***}$,$0.096^{***}$
Partner self-employed,$-0.019$,$-0.020$,$0.298^{***}$
Partner College,$0.576^{***}$,$0.729^{***}$,$0.215^{***}$


In [12]:
def join_regtable(coefs, stds):
    cols= coefs.keys()
    #     print(cols)
    New_tab = coefs[cols[0]]
    New_tab = pd.concat((New_tab.rename(cols[0][18:]),stds[cols[0]].rename(cols[0][18:]+'_std')),axis=1)
    for name in cols[1:]:
        New_tab = pd.concat((New_tab,coefs[name].rename(name[18:]),stds[name].rename(name[18:]+'_std')),axis=1)
    return New_tab

In [13]:
join_regtable(tab_dummies_f.T, tab_dummies_std_f.T)

,,_std,,_std,{2}$,{2}$_std,,_std,,_std,...,}$,}$_std,,_std,,_std,,_std,,_std
"Parents, 25th pctl",$-0.212$,(0.1490),$0.246^{***}$,(0.0132),$-0.007^{***}$,(0.0009),$-1.873^{***}$,(0.0510),$0.966^{***}$,(0.0441),...,$-0.401^{***}$,(0.0490),$-0.337^{***}$,(0.0741),$-0.172^{**}$,(0.0803),$0.004$,(0.0883),$0.585^{***}$,(0.0389)
"Parents, 50th pctl",$1.516^{***}$,(0.2298),$0.320^{***}$,(0.0173),$-0.005^{***}$,(0.0010),$-3.817^{***}$,(0.0722),$0.460^{***}$,(0.0548),...,$0.102$,(0.0908),$-0.267^*$,(0.1389),$-0.120$,(0.1575),$-0.206$,(0.1413),$1.766^{***}$,(0.0586)
"Parents, 75th pctl",$6.298^{***}$,(0.2888),$0.118^{***}$,(0.0192),$0.004^{***}$,(0.0011),$-4.302^{***}$,(0.1856),$-1.225^{***}$,(0.0564),...,$-0.085$,(0.0892),$0.128$,(0.1062),$0.564^{***}$,(0.1302),$0.230$,(0.1431),$2.696^{***}$,(0.0712)


In [ ]:

all_data = [tab_dummies_f.T,tab_dummies_f_21.T,tab_dummies_f_22.T]
all_stds = [tab_dummies_std_f.T,tab_dummies_std_f_21.T,tab_dummies_std_f_22.T]

One_tab = {}
for i in range(3):
    coefs, stds = all_data[i], all_stds[i]
    cols= coefs.keys()
#     print(cols)
    New_tab = coefs[cols[0]]
    New_tab = pd.concat((New_tab.rename(cols[0][18:]),stds[cols[0]].rename(cols[0][18:]+'_std')),axis=1)
    for name in cols[1:]:
        New_tab = pd.concat((New_tab,coefs[name].rename(name[18:]),stds[name].rename(name[18:]+'_std')),axis=1)
        
    One_tab[2020+i] = New_tab.T.copy()
    
Final_tab = pd.concat((One_tab[2020]['Parents, 25th pctl'].rename("2020"),
           One_tab[2021]['Parents, 25th pctl'].rename("2021"),
          One_tab[2022]['Parents, 25th pctl'].rename("2022")),axis=1)

for ind in Final_tab.index[1::2]:
    Final_tab.rename({ind:""}, inplace=True)
Final_tab.replace({np.nan:'--'}, inplace=True)

Final_tab2 = pd.concat((One_tab[2020]['Parents, 50th pctl'].rename("2020"),
           One_tab[2021]['Parents, 50th pctl'].rename("2021"),
          One_tab[2022]['Parents, 50th pctl'].rename("2022")),axis=1)

for ind in Final_tab2.index[1::2]:
    Final_tab2.rename({ind:""}, inplace=True)
Final_tab2.replace({np.nan:'--'}, inplace=True)

Final_tab3 = pd.concat((One_tab[2020]['Parents, 75th pctl'].rename("2020"),
           One_tab[2021]['Parents, 75th pctl'].rename("2021"),
          One_tab[2022]['Parents, 75th pctl'].rename("2022")),axis=1)

for ind in Final_tab3.index[1::2]:
    Final_tab3.rename({ind:""}, inplace=True)
Final_tab3.replace({np.nan:'--'}, inplace=True)

Final_final_tab = pd.concat((Final_tab,Final_tab2,Final_tab3),axis=1,keys=['25th percentile','50th percentile','75th percentile'])
print(Final_final_tab.to_latex(escape=False))
Final_final_tab.to_latex('./tables/qtregs_ind_3040_basic_text.tex',escape=False)

In [11]:
Btab.to_latex?

In [11]:
all_tabs = []

all_names = ['5','10','15']
formal_names = [ "Children younger than 5","Children younger than 10","Children younger than 15"]

for name in all_names:
    tab = Table.read('./regtabs/sqtreg_advanced_age3035_{}_covid.tex'.format(name)).to_pandas()
    tab = tab.set_index('col0')
    tab.rename(columns=dict(zip(tab.keys(),[i.replace('Parents,',"") for i in tab.keys()])),inplace=True)
    all_tabs.append(tab)
    
Btab = pd.concat(all_tabs,axis=1,keys=(formal_names))

Btab.index = [str(i).replace("nan","") for i in Btab.index]
# Btab.to_latex('./tables/sqtreg_advanced_age3035_all_covid.tex',escape=False,column_format='l|ccc|ccc|ccc',multicolumn_format='c')
Btab

Children younger than 5  \
                                                     25th pctl   
$\delta_{2020}$ x occ group 1                         $-0.246$   
                                                      (0.1809)   
$\delta_{2020}$ x occ group 2                          $0.306$   
                                                      (0.3648)   
$\delta_{2020}$ x occ group 3                    $0.687^{***}$   
...                                                        ...   
                                                      (0.2879)   
$\delta_{2022}$ x occ group 5 x female          $-0.361^{***}$   
                                                      (0.1214)   
N                                                      130,977   
Pseudo $R^2$                                            0.0353   

                                                                        \
                                             50th pctl       75th pctl   
$\delta_{2020}$ x occ group 1                 $-0.309$  $-1.645^{***}$   
                                              (0.2111)        (0.5313)   
$\delta_{2020}$ x occ group 2                 $-0.410$        $-0.781$   
                                              (0.4249)        (0.7141)   
$\delta_{2020}$ x occ group 3                  $0.282$        $-0.016$   
...                                                ...             ...   
                                              (0.5703)        (0.7177)   
$\delta_{2022}$ x occ group 5 x female  $-2.645^{***}$  $-1.681^{***}$   
                                              (0.5797)        (0.5556)   
N                                              130,977         130,977   
Pseudo $R^2$                                    0.0426          0.0291   

                                       Children younger than 10  \
                                                      25th pctl   
$\delta_{2020}$ x occ group 1                          $-0.249$   
                                                       (0.3055)   
$\delta_{2020}$ x occ group 2                           $0.090$   
                                                       (0.3826)   
$\delta_{2020}$ x occ group 3                     $0.692^{***}$   
...                                                         ...   
                                                       (0.1943)   
$\delta_{2022}$ x occ group 5 x female            $-0.343^{**}$   
                                                       (0.1716)   
N                                                       136,192   
Pseudo $R^2$                                             0.0355   

                                                                        \
                                             50th pctl       75th pctl   
$\delta_{2020}$ x occ group 1                 $-0.277$  $-1.532^{***}$   
                                              (0.2264)        (0.5133)   
$\delta_{2020}$ x occ group 2                 $-0.423$        $-0.857$   
                                              (0.4635)        (0.5847)   
$\delta_{2020}$ x occ group 3                  $0.306$         $0.131$   
...                                                ...             ...   
                                              (0.7332)        (0.8030)   
$\delta_{2022}$ x occ group 5 x female  $-2.871^{***}$   $-1.112^{**}$   
                                              (0.5309)        (0.4593)   
N                                              136,192         136,192   
Pseudo $R^2$                                    0.0446          0.0306   

                                       Children younger than 15  \
                                                      25th pctl   
$\delta_{2020}$ x occ group 1                          $-0.249$   
                                                       (0.1877)   
$\delta_{2020}$ x occ group 2                           $0.093$   
                                               

In [14]:
all_tabs = []

all_names = ['5','10','15']
formal_names = [ "Children younger than 5","Children younger than 10","Children younger than 15"]

for name in all_names:
    tab = Table.read('./regtabs/sqtreg_advanced_age3540_{}_covid.tex'.format(name)).to_pandas()
    tab = tab.set_index('col0')
    tab.rename(columns=dict(zip(tab.keys(),[i.replace('Parents,',"") for i in tab.keys()])),inplace=True)
    all_tabs.append(tab)
    
Btab = pd.concat(all_tabs,axis=1,keys=(formal_names))

Btab.index = [str(i).replace("nan","") for i in Btab.index]
Btab.to_latex('./tables/sqtreg_advanced_age3540_all_covid.tex',escape=False,column_format='l|ccc|ccc|ccc',multicolumn_format='c')
Btab

C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_7808\1293831293.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Btab.to_latex('./tables/sqtreg_advanced_age3540_all_covid.tex',escape=False,column_format='l|ccc|ccc|ccc',multicolumn_format='c')


Children younger than 5  \
                                                     25th pctl   
$\delta_{2020}$ x occ group 1                   $-0.998^{***}$   
                                                      (0.2864)   
$\delta_{2020}$ x occ group 2                         $-0.038$   
                                                      (0.2106)   
$\delta_{2020}$ x occ group 3                    $1.212^{***}$   
...                                                        ...   
                                                      (0.4155)   
$\delta_{2022}$ x occ group 5 x female          $-1.530^{***}$   
                                                      (0.3134)   
N                                                      250,030   
Pseudo $R^2$                                            0.0428   

                                                                        \
                                             50th pctl       75th pctl   
$\delta_{2020}$ x occ group 1           $-1.162^{***}$         $0.358$   
                                              (0.4407)        (0.2215)   
$\delta_{2020}$ x occ group 2                  $0.158$    $0.601^{**}$   
                                              (0.4078)        (0.2757)   
$\delta_{2020}$ x occ group 3            $1.131^{***}$        $-0.132$   
...                                                ...             ...   
                                              (0.9205)        (1.2962)   
$\delta_{2022}$ x occ group 5 x female  $-3.264^{***}$  $-2.267^{***}$   
                                              (0.5104)        (0.7575)   
N                                              250,030         250,030   
Pseudo $R^2$                                    0.0442          0.0233   

                                       Children younger than 10  \
                                                      25th pctl   
$\delta_{2020}$ x occ group 1                    $-1.021^{***}$   
                                                       (0.2144)   
$\delta_{2020}$ x occ group 2                          $-0.067$   
                                                       (0.1952)   
$\delta_{2020}$ x occ group 3                     $1.129^{***}$   
...                                                         ...   
                                                       (0.3196)   
$\delta_{2022}$ x occ group 5 x female           $-1.162^{***}$   
                                                       (0.2947)   
N                                                       273,851   
Pseudo $R^2$                                             0.0451   

                                                                       \
                                             50th pctl      75th pctl   
$\delta_{2020}$ x occ group 1           $-1.011^{***}$      $0.432^*$   
                                              (0.3093)       (0.2454)   
$\delta_{2020}$ x occ group 2                  $0.143$  $0.673^{***}$   
                                              (0.4201)       (0.2414)   
$\delta_{2020}$ x occ group 3            $1.150^{***}$       $-0.152$   
...                                                ...            ...   
                                              (0.6893)       (1.4684)   
$\delta_{2022}$ x occ group 5 x female  $-3.099^{***}$  $-2.203^{**}$   
                                              (0.6606)       (0.9235)   
N                                              273,851        273,851   
Pseudo $R^2$                                    0.0515         0.0288   

                                       Children younger than 15  \
                                                      25th pctl   
$\delta_{2020}$ x occ group 1                    $-1.015^{***}$   
                                                       (0.3071)   
$\delta_{2020}$ x occ group 2                          $-0.069$   
                                                       (0.17

In [15]:
all_tabs = []

all_names = ['5','10','15']
formal_names = [ "Children younger than 5","Children younger than 10","Children younger than 15"]

for name in all_names:
    tab = Table.read('./regtabs/sqtreg_advanced_age3540_{}_controls.tex'.format(name)).to_pandas()
    tab = tab.set_index('col0')
    tab.rename(columns=dict(zip(tab.keys(),[i.replace('Parents,',"") for i in tab.keys()])),inplace=True)
    all_tabs.append(tab)
    
Btab = pd.concat(all_tabs,axis=1,keys=(formal_names))

Btab.index = [str(i).replace("nan","") for i in Btab.index]
Btab.to_latex('./tables/sqtreg_advanced_age3540_all_controls.tex',escape=False,column_format='l|ccc|ccc|ccc',multicolumn_format='c')
Btab

C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_7808\2711105681.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Btab.to_latex('./tables/sqtreg_advanced_age3540_all_controls.tex',escape=False,column_format='l|ccc|ccc|ccc',multicolumn_format='c')


Children younger than 5                  \
                                         25th pctl       50th pctl   
$t$                                 $-0.014^{***}$  $-0.006^{***}$   
                                          (0.0011)        (0.0023)   
$t$ x female                         $0.020^{***}$   $0.043^{***}$   
                                          (0.0018)        (0.0030)   
unemployment rate x male             $0.036^{***}$   $0.025^{***}$   
                                          (0.0033)        (0.0068)   
unemployment rate x female           $0.046^{***}$  $-0.022^{***}$   
                                          (0.0042)        (0.0049)   
occ group 2                         $-0.609^{***}$   $0.248^{***}$   
                                          (0.0883)        (0.0727)   
occ group 3                         $-0.339^{***}$   $1.223^{***}$   
                                          (0.1301)        (0.0866)   
occ group 4                         $-2.464^{***}$  $-2.057^{***}$   
                                          (0.0856)        (0.0651)   
occ group 5                         $-3.942^{***}$  $-5.329^{***}$   
                                          (0.0755)        (0.0786)   
occ group 1 x female                       $0.021$    $0.247^{**}$   
                                          (0.1312)        (0.1242)   
occ group 2 x female                $-0.775^{***}$  $-0.446^{***}$   
                                          (0.0898)        (0.1359)   
occ group 3 x female                $-2.985^{***}$  $-3.710^{***}$   
                                          (0.1157)        (0.1638)   
occ group 4 x female                $-1.297^{***}$        $-0.215$   
                                          (0.0825)        (0.2030)   
occ group 5 x female                $-0.667^{***}$  $-0.997^{***}$   
                                          (0.0553)        (0.1325)   
$\beta_0$                            $4.342^{***}$   $8.504^{***}$   
                                          (0.0846)        (0.1040)   
N                                          250,030         250,030   
Pseudo $R^2$                                0.0428          0.0442   

                                           Children younger than 10  \
                                 75th pctl                25th pctl   
$t$                         $-0.013^{***}$           $-0.013^{***}$   
                                  (0.0019)                 (0.0008)   
$t$ x female                 $0.023^{***}$            $0.017^{***}$   
                                  (0.0027)                 (0.0016)   
unemployment rate x male    $-0.018^{***}$            $0.034^{***}$   
                                  (0.0063)                 (0.0025)   
unemployment rate x female  $-0.077^{***}$            $0.049^{***}$   
                                  (0.0038)                 (0.0039)   
occ group 2                  $1.535^{***}$           $-0.638^{***}$   
                                  (0.0594)                 (0.1266)   
occ group 3                  $2.557^{***}$           $-0.549^{***}$   
                                  (0.0776)                 (0.1096)   
occ group 4                  $0.513^{***}$           $-2.577^{***}$   
                                  (0.0478)                 (0.0885)   
occ group 5                 $-3.028^{***}$           $-3.981^{***}$   
                                  (0.1481)                 (0.0810)   
occ group 1 x female         $1.131^{***}$                 $-0.037$   
                                  (0.1047)                 (0.1093)   
occ group 2 x female               $0.052$           $-0.943^{***}$   
                                  (0.1256)                 (0.1298)   
occ group 3 x female        $-1.759^{***}$           $-2.988^{***}$   
                                  (0.1091)                 (0.0995)   
occ group 4 x female         $0.897^{***}$           $-1.277^{***}$   
           

In [16]:
all_tabs = []

all_names = ['5','10','15']
formal_names = [ "Children younger than 5","Children younger than 10","Children younger than 15"]

for name in all_names:
    tab = Table.read('./regtabs/sqtreg_advanced_age3035_{}_controls.tex'.format(name)).to_pandas()
    tab = tab.set_index('col0')
    tab.rename(columns=dict(zip(tab.keys(),[i.replace('Parents,',"") for i in tab.keys()])),inplace=True)
    all_tabs.append(tab)
    
Btab = pd.concat(all_tabs,axis=1,keys=(formal_names))

Btab.index = [str(i).replace("nan","") for i in Btab.index]
Btab.to_latex('./tables/sqtreg_advanced_age3035_all_controls.tex',escape=False,column_format='l|ccc|ccc|ccc',multicolumn_format='c')
Btab

C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_7808\2287887052.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Btab.to_latex('./tables/sqtreg_advanced_age3035_all_controls.tex',escape=False,column_format='l|ccc|ccc|ccc',multicolumn_format='c')


Children younger than 5                  \
                                         25th pctl       50th pctl   
$t$                                 $-0.008^{***}$  $-0.009^{***}$   
                                          (0.0011)        (0.0019)   
$t$ x female                         $0.005^{***}$   $0.027^{***}$   
                                          (0.0014)        (0.0030)   
unemployment rate x male             $0.035^{***}$   $0.072^{***}$   
                                          (0.0029)        (0.0057)   
unemployment rate x female           $0.062^{***}$   $0.061^{***}$   
                                          (0.0032)        (0.0060)   
occ group 2                          $-0.218^{**}$   $0.574^{***}$   
                                          (0.0985)        (0.1160)   
occ group 3                         $-0.659^{***}$   $0.435^{***}$   
                                          (0.1004)        (0.1614)   
occ group 4                         $-1.407^{***}$  $-0.954^{***}$   
                                          (0.0655)        (0.1003)   
occ group 5                         $-2.324^{***}$  $-3.404^{***}$   
                                          (0.0621)        (0.1227)   
occ group 1 x female                      $-0.105$         $0.229$   
                                          (0.1055)        (0.1485)   
occ group 2 x female                $-0.725^{***}$  $-0.755^{***}$   
                                          (0.0905)        (0.1265)   
occ group 3 x female                $-1.347^{***}$  $-1.943^{***}$   
                                          (0.1028)        (0.1240)   
occ group 4 x female                $-0.684^{***}$         $0.051$   
                                          (0.0922)        (0.2154)   
occ group 5 x female                $-0.506^{***}$  $-0.704^{***}$   
                                          (0.0460)        (0.0771)   
$\beta_0$                            $2.428^{***}$   $4.636^{***}$   
                                          (0.0662)        (0.1192)   
N                                          130,977         130,977   
Pseudo $R^2$                                0.0353          0.0426   

                                           Children younger than 10  \
                                 75th pctl                25th pctl   
$t$                          $0.018^{***}$           $-0.008^{***}$   
                                  (0.0017)                 (0.0011)   
$t$ x female                 $0.017^{***}$            $0.005^{***}$   
                                  (0.0024)                 (0.0012)   
unemployment rate x male     $0.021^{***}$            $0.035^{***}$   
                                  (0.0061)                 (0.0028)   
unemployment rate x female   $0.015^{***}$            $0.061^{***}$   
                                  (0.0050)                 (0.0029)   
occ group 2                  $1.469^{***}$            $-0.208^{**}$   
                                  (0.0901)                 (0.0951)   
occ group 3                  $1.779^{***}$           $-0.668^{***}$   
                                  (0.1294)                 (0.0946)   
occ group 4                  $1.170^{***}$           $-1.430^{***}$   
                                  (0.0810)                 (0.0516)   
occ group 5                 $-2.087^{***}$           $-2.323^{***}$   
                                  (0.1926)                 (0.0682)   
occ group 1 x female          $0.204^{**}$                 $-0.078$   
                                  (0.0999)                 (0.1070)   
occ group 2 x female        $-1.002^{***}$           $-0.769^{***}$   
                                  (0.0777)                 (0.0933)   
occ group 3 x female        $-1.479^{***}$           $-1.387^{***}$   
                                  (0.1299)                 (0.0969)   
occ group 4 x female            $-0.237^*$           $-0.730^{***}$   
           

In [20]:
all_tabs = []

all_names = ['mothers_age3040_10','mothers0k_age3040']
formal_names = [ "Mothers","Women without Children"]

for name in all_names:
    tab = Table.read('./regtabs/sqtreg_figure6_{}_agefix.tex'.format(name)).to_pandas()
    tab = tab.set_index('col0')
    tab.rename(columns=dict(zip(tab.keys(),[i.replace('Parents,',"") for i in tab.keys()])),inplace=True)
    all_tabs.append(tab)
    
Btab = pd.concat(all_tabs,axis=1,keys=(formal_names))

Btab.index = [str(i).replace("nan","") for i in Btab.index]
Btab.to_latex('./tables/sqtreg_figure6_allwomen_age3040_10_agefix.tex',escape=False,column_format='l|ccc|ccc',multicolumn_format='c')
Btab

C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_7808\1840227655.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Btab.to_latex('./tables/sqtreg_figure6_allwomen_age3040_10_agefix.tex',escape=False,column_format='l|ccc|ccc',multicolumn_format='c')


Mothers                  \
                                             25th pctl       50th pctl   
Tenure Partner                           $0.098^{***}$   $0.191^{***}$   
                                              (0.0019)        (0.0027)   
$t$                                      $0.035^{***}$   $0.047^{***}$   
                                              (0.0038)        (0.0057)   
$t^2$                                   $-0.001^{***}$  $-0.001^{***}$   
                                              (0.0001)        (0.0001)   
$t$ x female                             $0.038^{***}$        $-0.011$   
                                              (0.0054)        (0.0081)   
$t^2$ x female                          $-0.000^{***}$   $0.001^{***}$   
                                              (0.0001)        (0.0002)   
$\delta_{2020}$                          $0.362^{***}$        $-0.180$   
                                              (0.0734)        (0.1572)   
$\delta_{2021}$                          $0.718^{***}$         $0.152$   
                                              (0.1488)        (0.2225)   
$\delta_{2022}$                          $1.113^{***}$         $0.149$   
                                              (0.1481)        (0.3122)   
female                                  $-0.855^{***}$  $-1.173^{***}$   
                                              (0.0659)        (0.0992)   
$\delta_{2020}$ x female                $-0.311^{***}$   $-0.650^{**}$   
                                              (0.0975)        (0.2757)   
$\delta_{2021}$ x female                $-0.611^{***}$  $-1.781^{***}$   
                                              (0.1442)        (0.3221)   
$\delta_{2022}$ x female                $-0.852^{***}$  $-2.220^{***}$   
                                              (0.1743)        (0.4224)   
Part time x male                        $-1.817^{***}$  $-4.008^{***}$   
                                              (0.0472)        (0.0849)   
Part time x female                      $-0.913^{***}$  $-1.389^{***}$   
                                              (0.0224)        (0.0524)   
College x male                           $1.063^{***}$   $0.563^{***}$   
                                              (0.0362)        (0.0453)   
College x female                         $1.422^{***}$   $1.737^{***}$   
                                              (0.0312)        (0.0324)   
Less than High-school x male            $-0.760^{***}$  $-1.931^{***}$   
                                              (0.0686)        (0.1088)   
Less than High-school x female          $-0.423^{***}$  $-1.479^{***}$   
                                              (0.0351)        (0.0625)   
Partner self-employed x male                   $0.010$        $-0.024$   
                                              (0.0574)        (0.0730)   
Partner self-employed x female                $-0.015$   $0.174^{***}$   
                                              (0.0315)        (0.0447)   
Partner college x male                   $0.699^{***}$   $0.867^{***}$   
                                              (0.0423)        (0.0573)   
Partner college x female                 $0.599^{***}$   $0.494^{***}$   
                                              (0.0329)        (0.0395)   
Partner less than High-school x male    $-0.255^{***}$  $-0.838^{***}$   
                                              (0.0431)        (0.0983)   
Partner less than High-school x female  $-0.392^{***}$  $-1.044^{***}$   
                                              (0.0417)        (0.0799)   
Age $>$ 35                               $0.698^{***}$   $1.893^{***}$   
                                              (0.0216)        (0.0375)   
$\beta_0$                                $0.794^{***}$   $3.444^{***}$   
                                              (0.0412)        (0.0671)   
N                                              315,

In [21]:
all_tabs = []

all_names = ['mothers_age3040_10','mothers0k_age3040']
formal_names = [ "Mothers","Women without Children"]

for name in all_names:
    tab = Table.read('./regtabs/sqtreg_figure5_{}_agefix.tex'.format(name)).to_pandas()
    tab = tab.set_index('col0')
    tab.rename(columns=dict(zip(tab.keys(),[i.replace('Parents,',"") for i in tab.keys()])),inplace=True)
    all_tabs.append(tab)
    
Btab = pd.concat(all_tabs,axis=1,keys=(formal_names))

Btab.index = [str(i).replace("nan","") for i in Btab.index]
Btab.to_latex('./tables/ssqtreg_figure5_allwomen_age3040_10_agefix.tex',escape=False,column_format='l|ccc|ccc',multicolumn_format='c')
Btab

C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_7808\547750710.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Btab.to_latex('./tables/ssqtreg_figure5_allwomen_age3040_10_agefix.tex',escape=False,column_format='l|ccc|ccc',multicolumn_format='c')


Mothers                                  \
                                    25th pctl       50th pctl       75th pctl   
$\beta_0$                       $0.640^{***}$   $3.000^{***}$   $6.403^{***}$   
                                     (0.0652)        (0.1387)        (0.1970)   
Tenure Partner                  $0.191^{***}$   $0.319^{***}$   $0.197^{***}$   
                                     (0.0056)        (0.0076)        (0.0089)   
Tenure \ Partner$^{2}$         $-0.006^{***}$  $-0.007^{***}$  $-0.002^{***}$   
                                     (0.0003)        (0.0003)        (0.0004)   
Part Time                      $-0.920^{***}$  $-1.388^{***}$  $-1.048^{***}$   
                                     (0.0203)        (0.0398)        (0.0340)   
College                         $1.378^{***}$   $1.734^{***}$    $0.074^{**}$   
                                     (0.0402)        (0.0395)        (0.0340)   
Less than High-school          $-0.398^{***}$  $-1.375^{***}$  $-2.987^{***}$   
                                     (0.0479)        (0.0789)        (0.1651)   
Partner age                    $-0.012^{***}$  $-0.019^{***}$   $0.042^{***}$   
                                     (0.0018)        (0.0039)        (0.0054)   
Partner self-employed                 $0.026$   $0.225^{***}$   $0.190^{***}$   
                                     (0.0325)        (0.0413)        (0.0416)   
Partner College                 $0.528^{***}$   $0.366^{***}$   $-0.075^{**}$   
                                     (0.0376)        (0.0348)        (0.0361)   
Partner less than High-school  $-0.364^{***}$  $-0.974^{***}$  $-1.793^{***}$   
                                     (0.0257)        (0.0741)        (0.1326)   
$\delta_{2008-2011}$            $0.245^{***}$        $-0.040$  $-0.350^{***}$   
                                     (0.0289)        (0.0577)        (0.0686)   
$\delta_{2011-2014}$            $1.016^{***}$   $0.694^{***}$   $-0.156^{**}$   
                                     (0.0349)        (0.0609)        (0.0644)   
$\delta_{2014-2017}$            $1.096^{***}$   $1.308^{***}$   $0.416^{***}$   
                                     (0.0503)        (0.0523)        (0.0434)   
$\delta_{2017-2020}$            $0.372^{***}$   $1.297^{***}$   $0.692^{***}$   
                                     (0.0455)        (0.0721)        (0.0572)   
$\delta_{2020}$                 $0.281^{***}$   $0.836^{***}$   $0.886^{***}$   
                                     (0.0567)        (0.0922)        (0.0871)   
$\delta_{2021}$                       $0.059$         $0.159$   $0.433^{***}$   
                                     (0.0685)        (0.1400)        (0.0746)   
$\delta_{2022}$                      $-0.008$   $-0.249^{**}$         $0.080$   
                                     (0.0611)        (0.1182)        (0.1355)   
Age $>$ 35                      $0.758^{***}$   $1.957^{***}$   $3.006^{***}$   
                                     (0.0181)        (0.0261)        (0.0480)   
N                                     183,530         183,530         183,530   
Pseudo $R^2$                           0.0539          0.0778          0.0753   

                              Women without Children                  \
                                           25th pctl       50th pctl   
$\beta_0$                              $0.913^{***}$   $2.971^{***}$   
                                            (0.1327)        (0.1810)   
Tenure Partner                         $0.144^{***}$   $0.271^{***}$   
                                            (0.0053)        (0.0113)   
Tenure \ Partner$^{2}$                $-0.005^{***}$  $-0.008^{***}$   
                                            (0.0002)        (0.0005)   
Part Time                             $-1.230^{***}$  $-2.434^{***}$   
                                            (0.0367)        (0.0525)   
College                                $0.429^{***}$   $0.493^{***}$   
     

In [22]:
all_tabs = []

all_names = ['fathers_age3040_10','fathers0k_age3040']
formal_names = [ "Fathers","Men without Children"]

for name in all_names:
    tab = Table.read('./regtabs/sqtreg_figure5_{}_agefix.tex'.format(name)).to_pandas()
    tab = tab.set_index('col0')
    tab.rename(columns=dict(zip(tab.keys(),[i.replace('Parents,',"") for i in tab.keys()])),inplace=True)
    all_tabs.append(tab)
    
Btab = pd.concat(all_tabs,axis=1,keys=(formal_names))

Btab.index = [str(i).replace("nan","") for i in Btab.index]
Btab.to_latex('./tables/ssqtreg_figure5_allmen_age3040_10_agefix.tex',escape=False,column_format='l|ccc|ccc',multicolumn_format='c')
Btab

C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_7808\4132090700.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Btab.to_latex('./tables/ssqtreg_figure5_allmen_age3040_10_agefix.tex',escape=False,column_format='l|ccc|ccc',multicolumn_format='c')


Fathers                                  \
                                    25th pctl       50th pctl       75th pctl   
$\beta_0$                            $-0.212$   $1.516^{***}$   $6.298^{***}$   
                                     (0.1490)        (0.2298)        (0.2888)   
Tenure Partner                  $0.246^{***}$   $0.320^{***}$   $0.118^{***}$   
                                     (0.0132)        (0.0173)        (0.0192)   
Tenure \ Partner$^{2}$         $-0.007^{***}$  $-0.005^{***}$   $0.004^{***}$   
                                     (0.0009)        (0.0010)        (0.0011)   
Part Time                      $-1.873^{***}$  $-3.817^{***}$  $-4.302^{***}$   
                                     (0.0510)        (0.0722)        (0.1856)   
College                         $0.966^{***}$   $0.460^{***}$  $-1.225^{***}$   
                                     (0.0441)        (0.0548)        (0.0564)   
Less than High-school          $-0.653^{***}$  $-1.727^{***}$  $-2.587^{***}$   
                                     (0.0582)        (0.1165)        (0.2020)   
Partner age                     $0.032^{***}$   $0.061^{***}$   $0.096^{***}$   
                                     (0.0050)        (0.0080)        (0.0087)   
Partner self-employed                $-0.019$        $-0.020$   $0.298^{***}$   
                                     (0.0697)        (0.0993)        (0.0742)   
Partner College                 $0.576^{***}$   $0.729^{***}$   $0.215^{***}$   
                                     (0.0450)        (0.0666)        (0.0579)   
Partner less than High-school  $-0.190^{***}$  $-0.804^{***}$  $-1.822^{***}$   
                                     (0.0565)        (0.1414)        (0.1748)   
$\delta_{2008-2011}$                 $-0.069$         $0.041$  $-0.202^{***}$   
                                     (0.0518)        (0.0615)        (0.0729)   
$\delta_{2011-2014}$            $0.443^{***}$   $0.536^{***}$        $-0.118$   
                                     (0.0514)        (0.0718)        (0.0719)   
$\delta_{2014-2017}$            $-0.111^{**}$   $0.315^{***}$  $-0.216^{***}$   
                                     (0.0515)        (0.0764)        (0.0713)   
$\delta_{2017-2020}$           $-0.401^{***}$         $0.102$        $-0.085$   
                                     (0.0490)        (0.0908)        (0.0892)   
$\delta_{2020}$                $-0.337^{***}$      $-0.267^*$         $0.128$   
                                     (0.0741)        (0.1389)        (0.1062)   
$\delta_{2021}$                 $-0.172^{**}$        $-0.120$   $0.564^{***}$   
                                     (0.0803)        (0.1575)        (0.1302)   
$\delta_{2022}$                       $0.004$        $-0.206$         $0.230$   
                                     (0.0883)        (0.1413)        (0.1431)   
Age $>$ 35                      $0.585^{***}$   $1.766^{***}$   $2.696^{***}$   
                                     (0.0389)        (0.0586)        (0.0712)   
N                                     131,745         131,745         131,745   
Pseudo $R^2$                           0.0428          0.0619          0.0628   

                              Men without Children                  \
                                         25th pctl       50th pctl   
$\beta_0$                            $1.177^{***}$   $4.158^{***}$   
                                          (0.1157)        (0.1985)   
Tenure Partner                       $0.236^{***}$   $0.369^{***}$   
                                          (0.0078)        (0.0126)   
Tenure \ Partner$^{2}$              $-0.008^{***}$  $-0.009^{***}$   
                                          (0.0005)        (0.0013)   
Part Time                           $-1.489^{***}$  $-3.294^{***}$   
                                          (0.0425)        (0.0722)   
College                              $0.378^{***}$        $-0.111$   
                           